In [36]:
import pandas as pd
import numpy as np
from pathlib import Path 
import os
import datetime as dt
import requests
import json
from intuitlib.client import AuthClient
import urllib.parse
from urllib.parse import urlparse, unquote, urljoin
from time import perf_counter
import re
import yaml
from io import StringIO
from bs4 import BeautifulSoup

In [37]:
class Job:
    def __init__(self):
        base_dir = Path("c:/Users/ZheRao/OneDrive - Monette Farms/Monette Farms Team Site - Innovation Projects/Production/Database")
        self.json_download_path = Path("c:/Users/ZheRao/OneDrive - Monette Farms/Desktop/Work Files/Projects/5 - HP Data")
        self.base_dir = base_dir
        self.today = dt.date.today()
        month_format = "".join(["0",str(self.today.month)]) if self.today.month < 10 else str(self.today.month)
        self.us_companies = ["MFUSA", "MFAZ", "MSUSA", "MPUSA"]
        self.company_names = self.us_companies + ["MSL", "NexGen", "MFBC", "MPL", "MFL"]
        self.raw_path = {
            "QBO": {
                "Raw": base_dir/"Bronze"/"QBO"/"Raw"/f"{self.today.year}_{self.today.month}",
                "GL": base_dir/"Bronze"/"QBO"/"GeneralLedger",
                "PL": base_dir/"Bronze"/"QBO"/"ProfitAndLoss",
                "Time":base_dir/"Bronze"/"QBOTime",
                "APAR": base_dir/"Bronze"/"QBO"/"APAR"
            },
            "Delivery": {"Traction":base_dir/"Bronze"/"Traction", "HP":base_dir/"Bronze"/"HarvestProfit"},
            "Auth": {"QBO":base_dir/"Bronze"/"Authentication"/"QBO", "QBOTime": base_dir/"Bronze"/"Authentication"/"QBOTime",
                     "Harvest Profit": base_dir/"Bronze"/"Authentication"/"Harvest Profit"},
            "Log": base_dir/"Load_History"/f"{self.today.year}"/month_format
        }
        self.silver_path = {
            "QBO": {
                "Dimension_time": base_dir/"Silver"/"QBO"/"Dimension"/f"{self.today.year}_{self.today.month}",
                "Dimension": base_dir/"Silver"/"QBO"/"Dimension",
                "Raw": base_dir/"Silver"/"QBO"/"Fact"/"Raw",
                "PL": base_dir/"Silver"/"QBO"/"Fact"/"ProfitAndLoss",
                "GL": base_dir/"Silver"/"QBO"/"Fact"/"GeneralLedger",
                "Time": base_dir/"Silver"/"QBOTime",
                "APAR": base_dir/"Silver"/"QBO"/"Fact"/"APAR"
            },
            "Delivery": {"Traction":base_dir/"Silver"/"Traction", "HP":base_dir/"Silver"/"HarvestProfit"}
        }
        self.conversion_mt_to_lb = 2204.62262185 
        
    
    def get_fx(self):
        key  = os.getenv("ALPHAVANTAGE_KEY")
        url  = ("https://www.alphavantage.co/query?"
                "function=CURRENCY_EXCHANGE_RATE"
                "&from_currency=USD&to_currency=CAD"
                f"&apikey={key}")
        rate = float(requests.get(url, timeout=10).json()
                    ["Realtime Currency Exchange Rate"]["5. Exchange Rate"])
        self.fx = rate
    
    def create_log(self, path: Path) -> None:
        self.check_file(path)
        day_format = "".join(["0",str(self.today.day)]) if self.today.day < 10 else str(self.today.day)
        self.log = open(path/(day_format+"_Log.txt"), "a")

    
    def close_log(self):
        self.log.close()

    def check_file(self, path: Path) -> None:
        if not Path.exists(path):
            os.makedirs(path)
    
    def formulate_date(self, df:pd.DataFrame, date_cols:list[str], drop_time:bool=True) -> pd.DataFrame:
        """ 
            format the date columns into datetime format
        """
        assert len(set(date_cols) - set(df.columns)) == 0, "Not all columns passed are inside dataframe passed"
        for col in date_cols:
            df[col] = pd.to_datetime(df[col], utc=True)
            if drop_time: df[col] = df[col].dt.date
        return df





# QBOETL Class

# Project Class

In [38]:
class Projects(Job):
    """ 
        for project specific data transformations
    """
    
    def __init__(self, focus_last_FY:bool = False, is_dev:bool=False):
        super().__init__()
        self.gold_path = {
            "weekly_banking": self.base_dir / "Gold" / "FinanceProject" / "WeeklyBanking",
            "inventory": self.base_dir / "Gold" / "InventoryProject",
            "payroll": self.base_dir / "Gold" / "HRProject" /"PayrollProject",
            "finance_operational": self.base_dir / "Gold" / "FinanceOperationalProject",
            "budget": self.base_dir / "Gold" / "BudgetProject",
            "QBOTime": self.base_dir / "Gold" / "HRProject" / "QBOTimeProject",
            "hr_combined": self.base_dir / "Gold" / "HRProject" / "CombinedSummary",
            "pillar_dashboard": self.base_dir / "Gold" / "DirectorDashboards",
            "APReporting": self.base_dir / "Gold" / "FinanceProject" / "APReporting"
        }
        self.silver_acc = pd.read_csv(self.silver_path["QBO"]["Dimension_time"]/"Account.csv")
        self.commodities = {
            "Produce": ["Strawberry", "Watermelon", "Cantaloupe", "Market Garden", "Broccoli", "Pumpkin", "Sweet Corn", "Cauliflower", "Squash", "Honeydew Melon", "Potato", "Carrot", "Cabbage",
                        "Lettuce", "Brussel Sprouts", "Prairie Pathways", "Beet", "Corn Maze", "CSA"],
            "Grain": ["Blackeye Pea", "Winter Wheat", "Durum", "Cotton", "Chickpea", "Barley", "Green Lentil", "Red Lentil", "Canola", 
                        "Wheat","Field Pea", "Corn", "Oat", "Soybean", "Bean"],
            "Cattle": ["Weaned Calves", "Cull Bull", "Cull Cow", "Bred Heifer", "Purebred Yealing Bull", "Purebred Heifer", 
                        "Purebred Cow", "Purebred Bull", "Cow", "Bull", "Steer", "Heifer", "Yearling", "Calf"]
        }
        self.locations = {
            "Produce": ["BritishColumbia (produce)", "Outlook", "Arizona (produce)", "Montana (produce)", "Seeds USA"],
            "Cattle": ["Airdrie", "Eddystone (cattle)", "Ashcroft", "Home Ranch", "Diamond S", "Wolf Ranch", "Fraser River Ranch", "Moon Ranch", "Waldeck", "Calderbank"],
            "Grain": ["Eddystone (grain)", "Arizona (grain)", "Colorado", "Swift Current", "Regina", "Raymore", "Prince Albert", "The Pas",
                      "Kamsack", "Hafford", "Yorkton", "Fly Creek", "Camp 4", "Havre", "Billings"],
            "Seed": ["NexGen", "Seeds"],
            "Others": ["Eddystone (corporate)", "Arizona (corporate)", "Legacy", "BritishColumbia (corporate)", "Corporate"]
        }
        self.bc_ranches = ["Ashcroft", "Fraser River Ranch", "Moon Ranch", "Wolf Ranch", "Diamond S","Home Ranch"]
        self.pl_exist = False # determines whether _financial_operational has run and gold_pl is stored in self, if not, any subsequent downstream projects will run _financial_operational first
        self.currentFY = self.today.year if self.today.month<=10 else self.today.year + 1
        if focus_last_FY: self.currentFY -= 1
        self.is_dev = is_dev
        self.accnum_reroute = {"MFL405101": "MFL405110", "MFL405102":"MFL405120", "MFL405103":"MFL405130", "MSL585000":"MSL562505", "MSL402110": "MSL402112", "MFBC575000": "MFBC575020",
                               "MFBC629000": "MFBC629010"}
        self.accid_reroute = {"MFBC250": "MFBC210", "MFBC216":"MFBC210", "MFBC358":"MFBC210", "MFBC255":"MFBC210", "MFBC314":"MFBC272", "MFBC268":"MFBC584", "MFBC188":"MFBC192", "MFBC374":"MFBC190",
                              "MFBC61":"MFBC190", "MFBC26":"MFBC585", "MFBC412":"MFBC242", "MFBC66": "MFBC592", "MFBC65":"MFBC220", "MFBC19":"MFBC210", "MFBC60":"MFBC187"}

    def _pillar_classification(self, entry: pd.Series) -> str:
        """ 
            this function classifies pillar of a transaction based on location
        """
        location = entry["Location"]
        if not isinstance(location, str) or (location == "Missing"):
            return "Missing"
        if "produce" in location:
            return "Produce"
        elif "grain" in location:
            return "Grain"
        elif "cattle" in location:
            return "Cattle"
        elif "corporate" in location:
            return "Unclassified"
        match location.lower():
            case "hafford"|"kamsack"|"prince albert"|"raymore"|"regina"|"swift current"|"the pas"|"camp 4"|"fly creek"|"havre"|"yorkton"|"colorado"|"billings":
                return "Grain"
            case "outlook"|"seeds usa":
                return "Produce"
            case "ashcroft"|"diamond s"|"fraser river ranch"|"home ranch"|"moon ranch"|"wolf ranch"|"waldeck"|"calderbank"|"airdrie":
                return "Cattle"
            case "seeds"|"nexgen":
                return "Seed"
            case _:
                return "Unclassified"
    
    def _identify_product(self, entry: pd.Series, for_budget:bool=False) -> str:
        """ 
            this function identifies commodity from account names, except for seed, 
                if this function is called from budget project, it combines MG & CSA and take CM into consideration, and name forage differently
        """
        if not for_budget:
            if entry["Corp"] in ["MSL", "MSUSA", "NexGen"]:
                return "SeedProduct"
        accname = entry["AccName"].lower() if not for_budget else entry["AccFull"].lower()
        if "float" in accname:
            return "Others"
        for x in self.commodities["Produce"] + self.commodities["Grain"] + self.commodities["Cattle"]:
            if x.lower() in accname:
                if for_budget:
                    match x:
                        case "Market Garden"|"CSA":
                            return "Market Garden / CSA"
                        case "Corn Maze":
                            return "Prairie Pathways"
                    return x 
                return x
        if "straw" in accname or "forage" in accname or "hay bale" in accname:
            if for_budget: 
                return "Hay/Silage" 
            else: 
                return "Forage"
        return "Others"
    
    def _weekly_banking(self) -> None:
        """ 
            weekly banking project: match latest GL bank transactions with raw activities - extract accounts for those activities
                assumptions: a raw entry (e.g., invoice) can have multiple lines - multiple associated accounts, only considering the first one 
        """
        print("\nStarting Weekly Banking Project Transformation\n")
        # determine minal date to keep for GL
        if self.today.month > 6:
            year = self.today.year 
            month = self.today.month - 6 
        else:
            year = self.today.year - 1
            month = self.today.month + 12 - 6
        # load and prepare data
        ## account
        account = self.silver_acc.copy(deep=True)
        ## change some accounts to Transfer category
        acc_list = ["MFL264", "MSL250"]
        account.loc[account["AccID"].isin(acc_list), "ProfitType"] = "Asset"
        account.loc[account["AccID"].isin(acc_list), "Category"] = "Transfer"
        account_bank = account[account["AccountType"]=="Bank"]
        ## LinkedTxn for invoice and bill
        invoice_linked = pd.read_csv(self.silver_path["QBO"]["Raw"] / "LinkedTxn"/ "LinkedTxn_Mapping_Invoice.csv")
        bill_linked = pd.read_csv(self.silver_path["QBO"]["Raw"] / "LinkedTxn"/ "LinkedTxn_Mapping_Bill.csv")
        mapping = pd.concat([invoice_linked, bill_linked])
        mapping = mapping.drop(columns=["Corp"])
        # define customized function for processing other raw table
        def _process_facts(df_type:str) -> pd.DataFrame:
            """ 
                function for processing raw tables for mapping table - TransactionID_partial to AccID
            """
            df = pd.read_csv(self.silver_path["QBO"]["Raw"]/(df_type+".csv"), usecols = ["TransactionID", "AccID"])
            df["TransactionID"] = df["TransactionID"].apply(lambda x: x.split("-")[1])
            df = df.drop_duplicates()
            df = df.rename(columns={"TransactionID":"TxnId"})
            return df
        ## purchase table for expense transactions
        purchase = _process_facts("Purchase")
        purchase["TxnType"] = "Expense"
        mapping = pd.concat([mapping,purchase])
        ## journal entries - exclude most entries related to bank
        journal = _process_facts("JournalEntry")
        journal["TxnType"] = "Journal Entry"
        # for journal entries, exclude most of entires where the activity account ID is a bank ID
        exclude_list = list(account_bank.AccID.unique())
        # mylist = ["MFL51", "MFBC470", "MFBC471", "MFL28", "MFL27", "MFL1150040024"]
        mylist = ["MFBC470", "MFBC471"] # should include these accounts
        for acc in mylist:
            exclude_list.remove(acc)
        journal = journal[~journal["AccID"].isin(exclude_list)]
        mapping = pd.concat([mapping,journal])
        ## deposit
        deposit = _process_facts("Deposit")
        deposit["TxnType"] = "Deposit"
        mapping = pd.concat([mapping,deposit])
        ## salesreceipts
        sales = _process_facts("SalesReceipt")
        sales["TxnType"] = "Sales Receipt"
        mapping = pd.concat([mapping,sales])
        # process mapping table - dedup
        mapping = mapping.drop_duplicates(subset=["TxnId"],keep="first")
        ## load GL transacitons
        cols = ["TransactionType","TransactionID_partial","AccID","AccNum","AccName", "TransactionDate", "Amount", "SplitAcc", "SplitAccID", "Memo", "Corp", "Balance"]
        transactions = pd.read_csv(self.silver_path["QBO"]["GL"]/"GeneralLedger.csv",dtype={"TransactionID_partial":str}, usecols=cols)
        transactions = transactions[transactions["AccID"].isin(account_bank.AccID.unique())]
        transactions["TransactionDate"] = pd.to_datetime(transactions["TransactionDate"])
        transactions = transactions[transactions["TransactionDate"]>=dt.datetime(year, month, 1)]
        transactions = transactions.rename(columns={"TransactionType":"TxnType","TransactionID_partial":"TxnId",
                                                    "AccID":"BankAccID","AccNum":"BankAccNum","AccName":"BankAccName",
                                                    "TransactionDate":"BankActivityDate","Amount":"BankAmount"})
        transactions["Sign"] = transactions["BankAmount"].apply(lambda x: "Positive" if x>=0 else "Negative")
        # merge to get CurrencyID for bank_acc
        transactions = pd.merge(transactions, account_bank.loc[:,["AccID","CurrencyID"]], left_on=["BankAccID"], right_on=["AccID"], how="left")
        transactions = transactions.drop(columns=["AccID"])
        # separating transfers - don't merge with mapping table
        transfers = transactions[transactions["TxnType"] == "Transfer"].copy(deep=True)
        transactions = transactions[transactions["TxnType"]!="Transfer"]
        transactions = transactions.drop(columns=["SplitAcc", "SplitAccID"])
        transactions["BankActivityDate"] = pd.to_datetime(transactions["BankActivityDate"])
        transactions["TxnType"] = transactions["TxnType"].replace({"Cheque Expense":"Expense", "Check": "Expense"})
        # merge with mapping table
        transactions_mapped = pd.merge(transactions,mapping,on=["TxnId","TxnType"],how="left")
        non_match = transactions_mapped[transactions_mapped["AccID"].isna()]
        print("None Match Transaction Types")
        print(non_match.TxnType.value_counts())
        print(f"Non matches - {len(non_match)}")
        # function to determine transfer type
        def _determine_transfer_type(entry:str) -> str:
            """ 
                determine whether the transfer is for visa, bank, or other transfer
            """
            if "visa" in entry.lower():
                return "Visa Payment"
            elif "due" in entry.lower():
                return "Bank Transfer"
            else:
                return "Other Transfer"
        # allocate transfer type 
        transfers["TransferType"] = transfers["SplitAcc"].apply(lambda x: _determine_transfer_type(x))
        transfers = transfers.rename(columns={"SplitAccID":"AccID"})
        transfers = transfers.drop(columns=["SplitAcc"])
        transactions_mapped = pd.concat([transactions_mapped,transfers], ignore_index=True)
        # clean up the dataframe
        transactions_mapped = transactions_mapped.rename(columns={"CurrencyID":"BankCurrencyID"})
        transactions_mapped = pd.merge(transactions_mapped, account.loc[:,["AccID","AccName","AccNum","Category","ProfitType","CurrencyID"]], on="AccID", how="left")
        transactions_mapped.loc[transactions_mapped["TransferType"]=="Bank Transfer","Category"] = "Bank Transfer"
        transactions_mapped.loc[((transactions_mapped["BankAccNum"].str.startswith("MSL"))&(transactions_mapped["AccNum"]=="MSL120001")), "Category"] = "Seed Processing Revenue"
        transactions_mapped = transactions_mapped.rename(columns={"AccNum":"ActivityAccNum", "AccName":"ActivityAccName"})
        transactions_mapped.loc[((transactions_mapped["TxnType"]=="Sales Tax Payment")&(transactions_mapped["Sign"]=="Positive")), "ProfitType"] = "Other Operating Revenue"
        transactions_mapped.loc[((transactions_mapped["TxnType"]=="Sales Tax Payment")&(transactions_mapped["Sign"]=="Positive")), "Category"] = "Miscellaneous income"
        transactions_mapped.loc[((transactions_mapped["TxnType"]=="Sales Tax Payment")&(transactions_mapped["Sign"]=="Negative")), "ProfitType"] = "Operating Overheads"
        transactions_mapped.loc[((transactions_mapped["TxnType"]=="Sales Tax Payment")&(transactions_mapped["Sign"]=="Negative")), "Category"] = "Office and miscellaneous"
        transactions_mapped.loc[((transactions_mapped["TxnType"]=="Sales Tax Payment")), "ActivityAccNum"] = "Manual Adjustment"
        transactions_mapped.loc[((transactions_mapped["TxnType"]=="Sales Tax Payment")), "ActivityAccName"] = "Manual Adjustment"
        # csv from sharepoint is unstable, and produced unpredictable readings from Power BI
        self.check_file(self.gold_path["weekly_banking"])
        transactions_mapped.to_excel(self.gold_path["weekly_banking"]/"BankingActivity.xlsx", sheet_name="transactions", index=False)

    def _extract_accnum_accid(self) -> None:
        """ 
            this function creates a accnum to accID mapping table to avoid repeated merges
        """
        self.acc_map = self.operation_acc.set_index(["AccNum"])["AccID"]

    def _perform_manual_adjust_GL_inventory(self, df: pd.DataFrame) -> pd.DataFrame:
        """ 
            This function reads inventory account balances at the beginning of this fiscal year, and apply the amount to PL accounts for Fert/Chem/Seed
        """
        # fixed column values 
        date = '2024-11-01'
        transaction_type = 'Manual Adjustments'
        memo = 'Adjustments from Trial Balance from beginning of this fiscal year'
        FY = 2025
        month = 'November'
        # read adjustments csv file
        adjustments = pd.read_csv(self.gold_path["finance_operational"]/"ManualAdjustments"/"2025.csv",dtype={"Amount":float})
        # compute various Amount columns to match the other PL entries
        adjustments["AmountAdj"] = -adjustments["Amount"]
        adjustments["AmountCAD"] = adjustments.apply(lambda x: x["AmountAdj"] * self.fx if x["Currency"]=="USD" else x["AmountAdj"],axis=1)
        adjustments["AmountDisplay"] = -adjustments["AmountCAD"]
        adjustments["AccNum"] = adjustments.apply(lambda x: "".join(x["DisplayName"].split(" ")[:2]),axis=1)
        # create additional entries
        addition_df = df.head(0).copy(deep=True)
        row = {"TransactionDate":date, "TransactionType":transaction_type, "Memo":memo, "FiscalYear":FY, "Month":month, "FXRate": self.fx}
        for i in range(len(adjustments)):
            entry = row | {"Amount": adjustments.loc[i,"Amount"], "AccID": adjustments.loc[i,"AccID"], "AmountAdj": adjustments.loc[i,"AmountAdj"], 
                        "AmountCAD": adjustments.loc[i,"AmountCAD"], "AmountDisplay":adjustments.loc[i,"AmountDisplay"],
                        "Location":adjustments.loc[i,"Location"], "Pillar": adjustments.loc[i,"Pillar"],
                        "AccNum":adjustments.loc[i,"AccNum"] }
            addition_df.loc[len(addition_df)] = entry
        print(f"\nManual GL Inventory Accounts Adjustments created {len(addition_df)} entries\n")
        df = pd.concat([df,addition_df],ignore_index=True)
        return df

    def _finance_operational(self) -> None:
        """ 
            transform PL data into operational-ready
                1. reclassify accounts
                2. standardize location, classify pillar
                3. revising signs
        """
        print("\nStarting Finance Operational Project Transformation\n")
        # load data from silver space
        data = pd.read_csv(self.silver_path["QBO"]["PL"]/"ProfitAndLoss.csv")
        assert len(data.FXRate.value_counts()) == 1, "different FXRate detected"
        self.fx = data.loc[0,"FXRate"]
        data["TransactionDate"] = pd.to_datetime(data["TransactionDate"])
        data["FiscalYear"] = data.TransactionDate.apply(lambda x: x.year + 1 if x.month >= 11 else x.year)
        # add month to the PL
        data["Month"] = data["TransactionDate"].dt.month_name()
        ## add location for seed operation
        data.loc[data["Corp"]=="MSL","Location"] = "Seeds"
        data.loc[data["Corp"]=="NexGen","Location"] = "NexGen"
        data.loc[data["Corp"]=="MSUSA","Location"] = "Seeds USA"
        # clean location
        data = data.rename(columns={"Location":"LocationRaw"})
        data["Location"] = data["LocationRaw"]
        data = data.fillna(value={"Location":"Missing"})
        # switch seeds usa to AZ produce
        data.loc[data["Corp"]=="MSUSA","Location"] = "Arizona (produce)"
        ## clean location
        clean_location = {"Airdrie - Grain":"Airdrie", "Airdrie - Cattle":"Airdrie", "Airdrie - General":"Airdrie", "Airdrie":"Airdrie", 
                        "Eddystone - Grain": "Eddystone (grain)", "Eddystone - Cattle": "Eddystone (cattle)", "Eddystone - General":"Eddystone (corporate)",
                        "Outlook (JV)":"Outlook", "AZ Produce":"Arizona (produce)", "Corporate":"Arizona (corporate)", "BC Produce":"BritishColumbia (produce)",
                        "Grain":"Arizona (grain)", "Ashcroft (CC, Fischer, Loon)":"Ashcroft", 
                        "Outlook (Capital)":"Outlook", "Colorado (MF)":"Colorado", "Colorado (JV)":"Colorado", "Cattle - General":"BritishColumbia (corporate)",
                        "Home (70 M, LF/W, 105 M)":"Home Ranch", "Diamond S (BR)":"Diamond S", "-Corporate":"Corporate",
                        "MT Produce": "Montana (produce)"}
        others = {"North Farm (deleted)":"Legacy", "Cache/Fischer/Loon - DNU":"Legacy"}
        data["Location"] = data["Location"].replace(clean_location)
        locations = self.locations["Produce"] + self.locations["Grain"] + self.locations["Cattle"] + self.locations["Others"] + self.locations["Seed"]
        unaccounted_location = list(set(data["Location"].unique()) - set(locations))
        print(f"location unaccounted for - {unaccounted_location}")
        # classify pillar
        data["Pillar"] = data.apply(lambda x: self._pillar_classification(x),axis=1)
        # reorganize corp
        ## MPUSA missing location = Arizona (produce)
        data.loc[((data["Corp"] == "MPUSA")&(data["Location"].isna())), "Location"] = "Arizona (produce)"
        data.loc[((data["Corp"] == "MPUSA")&(data["Location"]=="Missing")), "Location"] = "Arizona (produce)"
        data.loc[((data["Corp"] == "MPUSA")&(data["Location"] == "Arizona (produce)")), "Pillar"] = "Produce"
        ## AZ Produce --> MPUSA
        data.loc[data["Location"] == "Arizona (produce)", "Corp"] = "MPUSA"
        ## move everything for AZ in 2024 to produce
        data.loc[((data["FiscalYear"] >= 2024) & (data["Location"].str.contains("Arizona",case=False))),"Pillar"] = "Produce"
        data.loc[((data["FiscalYear"] >= 2024) & (data["Location"].str.contains("Arizona",case=False))),"Location"] = "Arizona (produce)"
        ## BC Produce --> MPL
        data.loc[data["Location"] == "BritishColumbia (produce)", "Corp"] = "MPL"
        ## Outlook --> MPL
        data.loc[data["Location"]=="Outlook", "Corp"] = "MPL"
        # reroute accid
        data["AccID"] = data["AccID"].replace(self.accid_reroute)
        # Reclassify accounts for Operational Purpose
        ## read & process operational classification
        with open(self.silver_path["QBO"]["Dimension"]/"acc_classification.yaml", "r", encoding="utf-8") as f:
            raw_acc = yaml.safe_load(f)
        rows = [(l1, l2, l3, v) 
                for l1, l1_inner in raw_acc.items() 
                for l2, l2_inner in l1_inner.items() 
                for l3, l3_inner in l2_inner.items() 
                for v in l3_inner]
        acc_operation = pd.DataFrame(rows, columns=["OperationProfType", "OperationCategory", "OperationSubCategory", "AccID"])
        ## read accounts table and apply new classification
        accounts = self.silver_acc
        accounts = pd.merge(accounts, acc_operation, on = "AccID", how = "left")
        accounts["Commodity"] = accounts.apply(lambda x: self._identify_product(x), axis=1)
        commodities = pd.DataFrame(data={"Commodity": accounts["Commodity"].unique()})
        commodities.to_csv(self.gold_path["inventory"]/"Tables"/"commodities_acc.csv", index=False)
        # prepare account table for mapping AccID from AccNum
        self.operation_acc = accounts[accounts["AccNum"].notna()]   # AccNum must be non-missing
        self.operation_acc = self.operation_acc[self.operation_acc["Active"]] # avoid non-active accounts what share same AccNum with active accounts
        self.operation_acc.to_csv(self.gold_path["finance_operational"]/"AccNumTOAccID.csv", index=False)
        # Revising Signs according to Operational Classification
        print("Revising Signs ...")
        # expense_accounts = accounts[(accounts["OperationCategory"] == "Expense") | (accounts["OperationCategory"] =="Inventory Consumption")] # for my classification
        expense_accounts = accounts[accounts["ProfitType"].isin(["Cost of Goods Sold", "Direct Operating Expenses", "Operating Overheads", "Other Expense"])]
        data["AmountDisplay"] = data.apply(lambda x: -x["AmountCAD"] if x["AccID"] in expense_accounts.AccID.unique() else x["AmountCAD"], axis=1)
        data = self._perform_manual_adjust_GL_inventory(data)
        self.gold_pl = data
        self.gold_acc = accounts
        # save files
        print("Saving ...")
        self.check_file(self.gold_path["finance_operational"])
        data.to_csv(self.gold_path["finance_operational"]/"PL.csv", index=False)
        accounts.to_excel(self.gold_path["finance_operational"]/"Account_table.xlsx", sheet_name = "Account", index=False)
        data.to_excel(self.gold_path["finance_operational"]/"PL.xlsx", sheet_name="Transactions", index=False)
        for pillar in ["Grain", "Cattle", "Seed", "Produce"]:
            data[data["Pillar"]==pillar].to_excel(self.gold_path["pillar_dashboard"]/pillar/"PL.xlsx", sheet_name="Transactions", index=False)
        self.pl_exist = True
    
    def _process_pp(self, data:pd.DataFrame) -> pd.DataFrame:
        """ 
            This function takes original dataframe, apply the payperiod number classification based on transactions date, process payperiod columns, and return the new dataframe,
                save the pp table for consolidated tables
        """
        # load payperiods
        payperiods = pd.read_csv(self.gold_path["payroll"]/"Payperiods.csv")
        payperiods["START"] = pd.to_datetime(payperiods["START"])
        payperiods["END"] = pd.to_datetime(payperiods["END"])
        payperiods = payperiods.loc[:,["PP","START","END","Cycle","FiscalYear"]]
        def _determine_pp(entry:pd.Series, date_col:str = "TransactionDate") -> str:
            """ 
                This function determined which payperiod a transaction should be classified into, 
                    starting from most recent payperiod, the algorithm uses period start date + drift to determine which payperiod a transaction should fall into
                Assumptions:
                    1. for outlook and az, each payperiod is shifted by 5 + 7 days forward
                    2. for other location, each payperiod is shifted by 5 days forward
            """
            date = entry[date_col] 
            if isinstance(entry["Location"],str):
                location = entry["Location"].lower()
            else:
                location = "None"
            if "outlook" in location or "az" in location or "arizona" in location:
                date_diff = dt.timedelta(days=5+7)
            else:
                date_diff = dt.timedelta(days=5)
            year = date.year 
            month = date.month
            # push back the most recent payperiod dates for older transactions to save compute
            if month >= 11:
                payperiods_subset = payperiods[payperiods["END"] <= dt.datetime(year+1,2,1)] 
            else:  
                payperiods_subset = payperiods[payperiods["END"] <= dt.datetime(year,month+2,1)]
            for i in range(len(payperiods_subset)-1,-1,-1):
                if date > (payperiods_subset.loc[i,"END"]+date_diff):
                    return "Exceed Max PayPeriod"
                if date >= (payperiods_subset.loc[i,"START"]+date_diff):
                    return str(payperiods_subset.loc[i,"PP"]) + "-" + str(payperiods_subset.loc[i,"Cycle"]) + "-" + str(payperiods_subset.loc[i,"FiscalYear"])
            return "Earlier than Min PayPeriod"
        print("Allocating PPNum for transactions ...")
        date_col = "TransactionDate" if "TransactionDate" in data.columns else "date"
        data["PPNum"] = data.apply(lambda x: _determine_pp(x,date_col),axis=1)
        data = data[data["PPNum"] != "Earlier than Min PayPeriod"].copy(deep=True).reset_index(drop=True) # eliminate earlier than min payperiod in the csv, note dates are shifted in the csv
        data["Cycle"] = data["PPNum"].apply(lambda x: x.split("-")[1])
        data["FiscalYear"] = data["PPNum"].apply(lambda x: int(x.split("-")[2]))
        data["PPNum"] = data["PPNum"].apply(lambda x: x.split("-")[0])
        data["PPName"] = data["PPNum"].apply(lambda x: "PP0" + x if int(x) < 10 else "PP" + x)
        data["PPName"] = data["Cycle"].str.slice(2,) + "-" + data["PPName"]
        data.loc[:,["PPName", "PPNum", "Cycle", "FiscalYear"]].drop_duplicates().to_csv(self.gold_path["payroll"].parent/ "OtherTables" / "PayPeriods.csv", index=False)
        return data

    def _process_units(self) -> None:
        """ 
            this function read and process Unit files that contains unit numbers for each location
        """
        acres = pd.read_csv(self.gold_path["payroll"]/"Unit.csv",dtype={"Location":str, "Unit":float})
        acres["Location"] = acres["Location"].str.strip()
        doc_rename = {"Airdrie Grain": "Airdrie (grain)", "Aridrie Cattle (head days 365)":"Airdrie", "Arizona All":"Arizona (produce)",
                    "BC Cattle (head days 365)":"BritishColumbia (cattle)", "BC Produce":"BritishColumbia (produce)", 
                    "Box Elder":"Havre", "Eddystone Cattle (head days 365)":"Eddystone (cattle)", "Eddystone Grain":"Eddystone (grain)",
                    "Monette Seeds CDN (avg met. ton)":"Seeds", "Monette Seeds USA":"Seeds USA", "NexGen (avg met. ton)":"NexGen",
                    "Waldeck (head days 365)":"Waldeck", "Calderbank  (head days 365)":"Calderbank"}
        acres["Location"] = acres["Location"].replace(doc_rename)
        acres["Pillar"] = acres.apply(lambda x: self._pillar_classification(x),axis=1)
        acres.to_csv(self.gold_path["payroll"].parent/ "OtherTables" /"Unit_PowerBI.csv",index=False)

    def _payroll_project(self) -> None: 
        """ 
            will run _finance_operational() first
            output: details + cost per unit (units per location input sheet) + average cost per unit for FY
        """
        self.check_file(self.gold_path["payroll"].parent/ "OtherTables")
        print("\nStarting Payroll Project Transformation\n")

        # load and filter accounts for wages and contract labor
        account = self.silver_acc[(self.silver_acc["Category"].isin(["Wages and benefits - direct","Wages and benefits - overhead"]) | (self.silver_acc["AccNum"].isin(["MFAZ595001","MFBC536030"])))] 
        # load only with transaction date later than 2021-12-20, and without "Accrual" in the memo
        if self.is_dev:
            data = pd.read_csv(self.gold_path["finance_operational"]/"PL.csv")
        else:
            if not self.pl_exist: self._finance_operational()
            data = self.gold_pl.copy(deep=True)
        data = data[data["AccID"].isin(account.AccID.unique())]
        data["TransactionDate"] = pd.to_datetime(data["TransactionDate"])
        data = data[data["TransactionDate"]>=dt.datetime(2021,12,20)].reset_index(drop=True)
        data = data[~data["Memo"].str.contains("Accrual",case=False,na=False)]
        # allocating payperiods
        data = self._process_pp(data=data)
        # standardizing location
        # data.loc[data["Location"]=="Airdrie (corporate)", "Pillar"] = "Cattle"                # deprecated
        # data.loc[data["Location"]=="Airdrie (corporate)", "Location"] = "Airdrie (cattle)"    # deprecated
        data.loc[data["Location"]=="Eddystone (corporate)", "Pillar"] = "Unclassified"
        data.loc[data["Location"]=="Eddystone (corporate)", "Location"] = "Unassigned"
        data.loc[data["Location"]=="Legacy", "Location"] = "Unassigned"
        data.loc[(data["Location"].str.contains("corporate",case=False,na=False)&(data["Location"]!="BritishColumbia (corporate)")),"Location"] = "Corporate"
        ## move BC ranches into BC Cattle
        data.loc[(data["Location"].isin(self.bc_ranches+["BritishColumbia (corporate)"])), "Location"] = "BritishColumbia (cattle)"
        data.loc[data["Location"] == "BritishColumbia (cattle)", "Pillar"] = "Cattle"
        # summarizing data
        ## by Location per PP
        data_summarized = pd.DataFrame(data.groupby(["Location","PPName","Pillar","FiscalYear","Cycle","PPNum"]).agg({"AmountDisplay":"sum"}).reset_index(drop=False))
        assert len(data_summarized) == len(data.groupby(["Location","PPName"]).agg({"AmountDisplay":"sum"}).reset_index(drop=False)), "Duplicated value detected for per Location per PP calculation"
        ## join acres data for CostPerUnit compute
        print("Summarizing ...")
        acres = pd.read_csv(self.gold_path["payroll"].parent/ "OtherTables" /"Unit_PowerBI.csv",dtype={"Location":str, "Unit":float})
        acres = acres.loc[:,["Location", "Unit"]]
        ### create BC cattle total units
        total_bc = 0
        for l in self.bc_ranches+["BritishColumbia (corporate)"]:
            total_bc += acres.loc[acres["Location"]==l, "Unit"].item()
        acres.loc[acres["Location"]=="BritishColumbia (cattle)", "Unit"] = total_bc
        acres["Unit"] = acres["Unit"].replace({0: 1})
        print(f"Unaccounted location for Acres Doc: {set(acres.Location.unique()) - set(data_summarized.Location.unique())}")
        print(f"Unaccounted location for QBO Payroll: {set(data_summarized.Location.unique()) - set(acres.Location.unique())}")
        data_summarized = pd.merge(data_summarized, acres, on="Location", how="left")
        data_summarized["CostPerUnit"] = data_summarized["AmountDisplay"] / data_summarized["Unit"] * 26
        data_summarized["Count"] = 1
        ## by Location
        data_summarized2 = data_summarized.groupby(by=["Location","FiscalYear","Pillar"]).agg({"CostPerUnit":"mean", "Count":"sum"}).reset_index(drop=False)
        data_summarized2 = data_summarized2.rename(columns={"CostPerUnit":"Avg CostPerUnit"})
        assert len(data_summarized2) == len(data_summarized.groupby(by=["Location","FiscalYear"]).agg({"CostPerUnit":"mean"})), "Duplicated value detected for per Location calculation"
        ## by pillar
        data_summarized3 = data_summarized2.groupby(by=["FiscalYear","Pillar"]).agg({"Avg CostPerUnit":"mean", "Count":"sum"}).reset_index(drop=False)
        assert len(data_summarized3) == len(data_summarized.groupby(by=["Pillar","FiscalYear"]).agg({"CostPerUnit":"mean"})), "Duplicated value detected for per Pillar calculation"
        # saving
        print("Saving ...")
        self.check_file(self.gold_path["payroll"])
        data.to_excel(self.gold_path["payroll"]/"Payroll.xlsx", sheet_name="Payroll", index=False)
        self.check_file(self.gold_path["hr_combined"] / "CSV")
        data_summarized.to_csv(self.gold_path["hr_combined"]/ "CSV" / "payroll_summarized1.csv", index=False)
        data_summarized2.to_csv(self.gold_path["hr_combined"]/ "CSV" / "payroll_summarized2.csv", index=False)
        data_summarized3.to_csv(self.gold_path["hr_combined"]/ "CSV" / "payroll_summarized3.csv", index=False)

    def _QBOTime_project(self) -> None:
        """ 
            apply PP allocation to QBO Time data, clean locaiton, and join relevant info into one table
        """
        print("\nStarting QBO Time Project Transformation\n")
        # read files
        timesheets = pd.read_csv(self.silver_path["QBO"]["Time"]/"timesheets.csv")
        jobcode = pd.read_csv(self.silver_path["QBO"]["Time"]/"jobcodes.csv")
        users = pd.read_csv(self.silver_path["QBO"]["Time"]/"users.csv")
        group = pd.read_csv(self.silver_path["QBO"]["Time"]/"group.csv")
        print(f"Read {len(timesheets)} timesheet records, {len(jobcode)} jobcodes, {len(users)} users, {len(group)} groups")
        timesheets_len, users_len = len(timesheets), len(users)
        # clean up location in group table
        ## Arizona - all produce
        group.loc[((group["corp_short"]=="A")&(group["location_name"]=="Monette Farms AZ")), "Location"] = "Arizona (produce)"
        group.loc[((group["corp_short"]=="A")&(group["location_name"]=="Monette Produce USA")), "Location"] = "Arizona (produce)"
        group.loc[((group["corp_short"]=="A")&(group["location_name"]=="Monette Seeds USA")), "Location"] = "Arizona (produce)"
        ## BC
        group.loc[((group["corp_short"]=="BC")&(group["location_name"]=="Ashcroft Ranch")), "Location"] = "Ashcroft"
        group.loc[((group["corp_short"]=="BC")&(group["location_name"]=="Cache/Fischer/Loon")), "Location"] = "BritishColumbia (cattle)"
        group.loc[((group["corp_short"]=="BC")&(group["location_name"].str.contains("silage", case=False))), "Location"] = "BritishColumbia (cattle)"
        group.loc[((group["corp_short"]=="BC")&(group["location_name"]=="Diamond S Ranch")), "Location"] = "Diamond S"
        group.loc[((group["corp_short"]=="BC")&(group["location_name"]=="Fraser River Ranch")), "Location"] = "Fraser River Ranch"
        group.loc[((group["corp_short"]=="BC")&(group["location_name"]=="Home Ranch (70 Mile, LF/W, BR)")), "Location"] = "Home Ranch"
        group.loc[((group["corp_short"]=="BC")&(group["location_name"]=="Moon Ranch")), "Location"] = "Moon Ranch"
        group.loc[((group["corp_short"]=="BC")&(group["location_name"]=="Produce")), "Location"] = "BritishColumbia (produce)"
        group.loc[((group["corp_short"]=="BC")&(group["location_name"]=="Wolf Ranch")), "Location"] = "Wolf Ranch"
        group.loc[((group["corp_short"]=="BC")&(group["location_name"]=="SAWP")), "Location"] = "BritishColumbia (produce)"
        group.loc[((group["corp_short"]=="BC")&(group["location_name"]=="SAWP Produce")), "Location"] = "BritishColumbia (produce)"
        ## Outlook
        group.loc[((group["corp_short"]=="O")), "Location"] = "Outlook"
        ## others
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Airdrie")), "Location"] = "Airdrie"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="BC")), "Location"] = "Unassigned"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Calderbank")), "Location"] = "Calderbank"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Eddystone")), "Location"] = "Eddystone (unspecified)"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Hafford")), "Location"] = "Hafford"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Kamsack")), "Location"] = "Kamsack"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="MFUSA Billings")), "Location"] = "Billings"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="MFUSA Box Elder")), "Location"] = "Havre"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Nexgen Seeds")), "Location"] = "NexGen"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Prince Albert")), "Location"] = "Prince Albert"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Raymore")), "Location"] = "Raymore"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Regina")), "Location"] = "Regina"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Russel Approvals")), "Location"] = "Unassigned"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Seeds")), "Location"] = "Seeds"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Swift Current")), "Location"] = "Swift Current"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="The Pas")), "Location"] = "The Pas"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Waldeck")), "Location"] = "Waldeck"
        unclassified = group[group["Location"].isna()].location_name.unique()
        if len(unclassified) > 0: print(f"\nUnclassified location - {unclassified}\n")
        # create another location column for general location where bc ranches are merged into one
        group = group.rename(columns={"Location": "Location (detail)"})
        group["Location"] = group["Location (detail)"]
        group.loc[(group["Location (detail)"].isin(self.bc_ranches+["BritishColumbia (corporate)"])), "Location"] = "BritishColumbia (cattle)"
        # merge tables into one table
        ## merge location into users
        users = pd.merge(users, group.loc[:,["group_id", "location_name", "Location", "Location (detail)"]].drop_duplicates(), on="group_id", how="left")
        ## merge users into timesheets
        timesheets = pd.merge(timesheets,users.loc[:,["user_id", "group_id", "username", "full_name", "location_name","Location","Location (detail)"]], on="user_id", how="left")
        ## merge job into timesheets
        timesheets = pd.merge(timesheets, jobcode.loc[:,["jobcode_id","job_name","type"]].rename(columns={"type":"job_type"}), on="jobcode_id", how="left")
        assert (len(users) == users_len) and (len(timesheets) == timesheets_len), f"duplicated records found, timesheets - {timesheets_len} vs {len(timesheets)}; users - {users_len} vs {len(users)}"
        # classify payperiods
        timesheets["date"] = pd.to_datetime(timesheets["date"])
        timesheets = self._process_pp(data=timesheets)
        # modify location for BC0
        timesheets.loc[timesheets["user_id"] == "BC6107856", "Location"] = "Unassigned"
        # classify pillars
        timesheets["Pillar"] = timesheets.apply(lambda x: self._pillar_classification(x), axis=1)
        timesheets.loc[timesheets["Pillar"] == "Missing", "Pillar"] = "Unclassified"
        # summarizing data
        ## by Location per PP 
        summarized = timesheets.groupby(["Location","PPName","FiscalYear","Cycle","PPNum", "Pillar"]).agg({"duration":"sum"}).reset_index(drop=False)
        assert len(summarized) == len(timesheets.groupby(["Location","PPName"]).agg({"duration":"sum"})), "duplicated value detected for timsheet per Location per PP summarization"
        ## read units file
        acres = pd.read_csv(self.gold_path["payroll"].parent/ "OtherTables" /"Unit_PowerBI.csv",dtype={"Location":str, "Unit":float})
        acres = acres.loc[:,["Location", "Unit"]]
        # addition = pd.DataFrame(data={"Location":["Billings"], "Unit":[acres[acres["Location"].isin(['Fly Creek', 'Camp 4'])].Unit.sum()]})
        # acres = pd.concat([acres,addition],ignore_index=True)
        print(f"Unaccounted location for Acres Doc: {set(acres.Location.unique()) - set(summarized.Location.unique())}")
        print(f"Unaccounted location for timesheets: {set(summarized.Location.unique()) - set(acres.Location.unique())}")
        ### create BC cattle + Billings total units
        total_bc = 0
        for l in self.bc_ranches+["BritishColumbia (corporate)"]:
            total_bc += acres.loc[acres["Location"]==l, "Unit"].item()
        acres.loc[acres["Location"]=="BritishColumbia (cattle)", "Unit"] = total_bc
        acres.loc[acres["Location"]=="Billings", "Unit"] = acres[acres["Location"].isin(["Fly Creek", "Camp 4"])].Unit.sum()
        acres["Unit"] = acres["Unit"].replace({0: 1})
        ## merge with units file
        summarized = pd.merge(summarized, acres, on="Location", how="left")
        ## calculate hours per unit
        summarized["HoursPerUnit"] = summarized["duration"] / summarized["Unit"] * 26
        summarized["Count"] = 1
        # summarize per location
        summarized2 = summarized.groupby(by=["Location","FiscalYear", "Pillar"]).agg({"HoursPerUnit":"mean", "Count":"sum"}).reset_index(drop=False)
        summarized2 = summarized2.rename(columns={"HoursPerUnit":"Avg HoursPerUnit"})
        assert len(summarized2) == len(timesheets.groupby(["Location","FiscalYear"]).agg({"duration":"sum"})), "duplicated value detected for timsheet per Location summarization"
        # summarize per pillar
        summarized3 = summarized2.groupby(by=["FiscalYear", "Pillar"]).agg({"Avg HoursPerUnit":"mean", "Count":"sum"}).reset_index(drop=False)
        assert len(summarized3) == len(timesheets[timesheets["Pillar"]!="Missing"].groupby(["Pillar","FiscalYear"]).agg({"duration":"sum"})), "duplicated value detected for timsheet per Pillar summarization"

        # saving
        print("Saving ...\n")
        self.check_file(self.gold_path["QBOTime"])
        timesheets.to_excel(self.gold_path["QBOTime"]/"QBOTime.xlsx", sheet_name = "QBOTime", index=False)
        self.check_file(self.gold_path["hr_combined"]/ "CSV")
        summarized.to_csv(self.gold_path["hr_combined"]/ "CSV" / "time_summarized1.csv", index=False)
        summarized2.to_csv(self.gold_path["hr_combined"]/ "CSV" / "time_summarized2.csv", index=False)
        summarized3.to_csv(self.gold_path["hr_combined"]/ "CSV" / "time_summarized3.csv", index=False)

    def _hr_summary(self) -> None:
        """ 
            This function consolidate payroll and QBO time summaries into one table for consolidated insights
        """
        final_df = [pd.DataFrame(), pd.DataFrame(), pd.DataFrame()]
        for i in [1, 2, 3]:
            payroll = pd.read_csv(self.gold_path["hr_combined"] / "CSV" / f"payroll_summarized{i}.csv")
            payroll_rename = {"AmountDisplay": "TotalAmount", "CostPerUnit": "AmountPerUnit", "Avg CostPerUnit": "Avg AmountPerUnit"}
            payroll = payroll.rename(columns=payroll_rename)
            payroll["Mode"] = "Payroll"
            time = pd.read_csv(self.gold_path["hr_combined"] / "CSV" / f"time_summarized{i}.csv")
            time_rename = {"duration": "TotalAmount", "HoursPerUnit": "AmountPerUnit", "Avg HoursPerUnit": "Avg AmountPerUnit"}
            time = time.rename(columns=time_rename)
            time["Mode"] = "Hours"
            final_df[i-1] = pd.concat([payroll, time], ignore_index=True)
        final_df[0].to_excel(self.gold_path["hr_combined"]/"Summarized.xlsx", sheet_name="Summarized", index=False)
        final_df[1].to_excel(self.gold_path["hr_combined"]/"Summarized2.xlsx", sheet_name="Summarized2", index=False)
        final_df[2].to_excel(self.gold_path["hr_combined"]/"Summarized3.xlsx", sheet_name="Summarized3", index=False)

    def _temp_get_product(self, entry:str) -> str:
        """ 
            temporary function for aligning product classification with Traction for QBO accounts, will change for HP
        """
        entry = entry.lower()
        if "durum" in entry:
            return "Durum"
        elif "wheat" in entry:
            return "Wheat"
        elif "canola" in entry:
            return "Canola"
        elif ("chickpea" in entry) or ("garbanzo bean" in entry):
            return "Chickpeas"
        elif ("peas" in entry) or ("field pea" in entry):
            return "Peas"
        elif "barley" in entry:
            return "Barley"
        elif "green lentil" in entry:
            return "Green Lentils"
        elif "red lentil" in entry:
            return "Red Lentils"
        elif "oats" in entry:
            return "Oats"
        elif "corn" in entry:
            return "Corn"
        else:
            return "Others" 

    def _raw_inventory(self) -> None:
        """ 
            prepare the data from raw QBO table for inventory project: only extracting partial Invoice, SalesReceipt, and Journal Entry
        """
        print("\nStarting Inventory Project Transformation ...\n")
        corps = ["MFL", "MFUSA"]
        cols = ["TransactionDate", "TransactionType", "TransactionID", "Corp", "Qty", "AccID", "FarmID", "CustomerID",
                "DocNumber", "TransactionEntered", "Amount"]
        journal_cols = [col for col in cols if col != "Qty"]
        # read tables
        print("Loading raw tables ...")
        account = self.silver_acc.copy(deep=True)
        account = account[account["Corp"].isin(corps)]
        account = account[account["AccountType"] == "Income"]
        farm = pd.read_csv(self.silver_path["QBO"]["Dimension_time"]/"Farm.csv")
        farm = farm[farm["Corp"].isin(corps)]
        customer = pd.read_csv(self.silver_path["QBO"]["Dimension_time"]/"Customer.csv")
        customer = customer[customer["Corp"].isin(corps)]
        first_date = dt.datetime(2023,11,1)
        invoice = pd.read_csv(self.silver_path["QBO"]["Raw"]/"Invoice.csv")
        invoice = invoice[invoice["Corp"].isin(corps)]
        invoice["TransactionDate"] = pd.to_datetime(invoice["TransactionDate"])
        invoice = invoice[invoice["TransactionDate"]>=first_date]
        invoice = invoice[invoice["AccID"].isin(account.AccID.unique())]
        sales = pd.read_csv(self.silver_path["QBO"]["Raw"]/"SalesReceipt.csv")
        sales = sales[sales["Corp"].isin(corps)]
        sales["TransactionDate"] = pd.to_datetime(sales["TransactionDate"])
        sales = sales[sales["TransactionDate"]>=first_date]
        sales = sales[sales["AccID"].isin(account.AccID.unique())]
        journal = pd.read_csv(self.silver_path["QBO"]["Raw"]/"JournalEntry.csv", dtype={"FarmID":str, "ClassID":str, "CustomerID":str, "EmployeeID":str}, usecols=journal_cols)
        journal = journal[journal["AccID"].isin(account.AccID.unique())]
        journal["TransactionDate"] = pd.to_datetime(journal["TransactionDate"])
        journal = journal[journal["TransactionDate"]>=first_date]
        journal = journal[~journal["TransactionEntered"].str.contains("Delivered and not settled", na=False)]
        journal = journal[~journal["TransactionEntered"].str.contains("Grain Inventory Receivable Adjustment", na=False)]
        # combining tables
        print("Combining Fact Tables ...")
        invoice = invoice.loc[:,[col for col in cols if col in invoice.columns]]
        sales = sales.loc[:,[col for col in cols if col in sales.columns]]
        journal = journal.loc[:,[col for col in cols if col in journal.columns]]
        facts = pd.concat([invoice, sales, journal], ignore_index=True)
        del invoice, sales, journal
        # join facts with dimension tables
        facts = pd.merge(facts, account.loc[:,["AccID","AccNum","AccName","Category","Subcategory"]], on=["AccID"], how="left")
        facts = pd.merge(facts, farm.loc[:,["FarmID","FarmName"]], on=["FarmID"], how="left")
        facts = pd.merge(facts, customer.loc[:,["CustomerID","CustomerName"]], on=["CustomerID"], how="left")
        facts = facts[facts["Subcategory"]=="Grain - cash settlements"]
        print(f"Total Fact Entries - {len(facts)}")
        # product column
        facts["Product"] = facts["AccName"].apply(lambda x: self._temp_get_product(x))
        # saving file
        print("Saving Files ...")
        self.check_file(self.gold_path["inventory"])
        facts.to_excel(self.gold_path["inventory"]/"Excel"/"QBO_Grain_Settlements.xlsx", sheet_name="settlement", index=False)
        print("Finished\n")

    def _buget_process_input(self, inputdata_path:Path, processed_path:Path) -> None:
        """ 
            this function processes and saves budget totals for production, input (chem/fert/seed), produce budgets, and JD Lease
        """
        ## commodity prices - everything is CAD except Winter Wheat is converted to USD
        pricing = pd.read_csv(inputdata_path/"25-Grain-Pricing.csv")
        pricing.loc[pricing["Commodity"]=="WW", "ForecastPrice"] *= self.fx
        ## production budget
        budget_production = pd.read_csv(inputdata_path/"25-Grain-Revenue.csv")
        budget_production = budget_production.melt(
            id_vars=["Location", "Currency", "Type"],
            var_name="Commodity",
            value_name = "Amount"
        )
        budget_production = budget_production.fillna(value = {"Amount": 0})
        budget_production["Commodity"] = budget_production["Commodity"].replace({"Hay/Silage":"Hay"})
        budget_production.loc[((budget_production["Location"]=="Airdrie")&(budget_production["Commodity"]=="Hay")), "Commodity"] = "Silage" # only Airdrie has silage, others have hay
        budget_production_summary = pd.DataFrame(budget_production.groupby(["Location","Currency","Commodity"]).agg({"Amount": "prod"})).reset_index(drop=False)
        budget_production_summary = budget_production_summary.rename(columns={"Amount":"TotalYield"})
        ### merge yield with commodity price to calculate forecast production value of commodities
        budget_production_summary = pd.merge(budget_production_summary,pricing,on=["Commodity"], how="left")
        ### manual adjustments to prices
        budget_production_summary.loc[((budget_production_summary["Location"] == "Airdrie") & (budget_production_summary["Commodity"] == "Hay")), "ForecastPrice"] = 85
        budget_production_summary.loc[((budget_production_summary["Location"] == "Colorado (Genoa)") & (budget_production_summary["Commodity"] == "WW")), "ForecastPrice"] = 13.75
        budget_production_summary.loc[budget_production_summary["Location"] == "Yorkton", "ForecastPrice"] *= 2/3
        budget_production_summary["ForecastProductionCAD"] = budget_production_summary["TotalYield"] * budget_production_summary["ForecastPrice"]
        budget_production_summary = budget_production_summary[budget_production_summary["ForecastProductionCAD"].notna()]
        budget_production_summary = budget_production_summary[budget_production_summary["ForecastProductionCAD"]!=0]
        ### convert prices back to USD for a adjusted column
        budget_production_summary["ForecastProductionAdj"] = budget_production_summary.apply(lambda x: x["ForecastProductionCAD"] / self.fx if x["Currency"] == "USD" else x["ForecastProductionCAD"],axis=1)
        ### save production budget
        budget_production_summary.to_csv(processed_path/"budget_production.csv",index=False)
        ## input budget
        input_budget = pd.read_csv(inputdata_path/"25-Input-Budget.csv")
        input_budget = input_budget.drop(columns=["Total acres"])
        input_budget = input_budget.melt(
            id_vars = ["Location", "Type"],
            var_name = "Commodity",
            value_name = "Amount"
        )
        input_budget = input_budget.fillna(value = {"Amount": 0})
        input_budget.loc[((input_budget["Location"]=="Yorkton")&(input_budget["Type"].isin(["Fertilizer","Chemical","Seed"]))), "Amount"] *= 2/3
        input_budget.to_csv(processed_path/"input_budget.csv",index=False)
        ## labour budget
        labour_budget = pd.read_csv(inputdata_path/"25-Labour-Budget.csv")
        labour_budget = labour_budget.melt(
            id_vars = ["Location","Currency"],
            var_name = "Month",
            value_name = "LabourBudgetCAD"
        )
        labour_budget["LabourBudgetAdj"] = labour_budget.apply(lambda x: x["LabourBudgetCAD"]/self.fx if x["Currency"]=="USD" else x["LabourBudgetCAD"], axis=1)
        labour_budget.to_csv(processed_path/"labour_budget.csv",index=False)
        ## outlook budget
        outlook = pd.read_csv(inputdata_path/"25-Outlook-Detail.csv")
        outlook = outlook.melt(
            id_vars=["Type", "ProfitType"],
            var_name="Commodity",
            value_name="Amount"
        )
        outlook = outlook.fillna(value={"Amount": 0})
        outlook.to_csv(processed_path/"outlook_budget.csv", index=False)
        ## AZ budget
        az = pd.read_csv(inputdata_path / "25-AZ-Detail.csv")
        az = az.melt(
            id_vars=["Type", "ProfitType"],
            var_name="CommodityRaw",
            value_name="AmountCAD"
        )
        az = az.fillna(value={"AmountCAD": 0})
        az.to_csv(processed_path/"az_budget.csv", index=False)
        ## BC produce details
        bc = pd.read_csv(inputdata_path / "25-BC-Detail.csv")
        bc = bc.melt(
            id_vars=["Type", "ProfitType"],
            var_name="CommodityRaw",
            value_name="AmountCAD"
        )
        bc = bc.fillna(value={"AmountCAD": 0})
        bc.to_csv(processed_path/"bc_budget.csv", index=False)
        ## JD lease
        jdlease = pd.read_csv(inputdata_path/"25-JD-Lease-Summary.csv")
        jdlease = jdlease[jdlease["AllocatedCost25"] != 0]
        jdlease.to_csv(processed_path/"JD_lease.csv", index=False)

    def _budget_read_outsidedata(self,processed_path:Path) -> tuple[pd.DataFrame,pd.DataFrame,pd.DataFrame,pd.DataFrame,pd.DataFrame,pd.DataFrame,pd.DataFrame]:
        """ 
            this function reads all the processed outside data and standardize the commodity and location naming
        """
        production_budget = pd.read_csv(processed_path/"budget_production.csv")
        input_budget = pd.read_csv(processed_path/"input_budget.csv")
        labour_budget = pd.read_csv(processed_path/"labour_budget.csv")
        outlook_budget = pd.read_csv(processed_path/"outlook_budget.csv")
        jdlease = pd.read_csv(processed_path/"JD_lease.csv")
        az_budget = pd.read_csv(processed_path/"az_budget.csv")
        bc_budget = pd.read_csv(processed_path/"bc_budget.csv")
        ## standardizing commodity naming
        production_rename_commodity = {"R Lentils":"Red Lentil", "G Lentils":"Green Lentil","Chickpeas":"Chickpea","Peas":"Field Pea", "WW": "Winter Wheat"}
        input_rename_commodity = {"R Lentils":"Red Lentil", "G Lentils":"Green Lentil","Chickpeas":"Chickpea", "WW": "Winter Wheat"}
        outlook_rename_commodity = {"Broccoli-cases/ac":"Broccoli", "Cabbage-lbs/ac":"Cabbage", "Carrots-lbs":"Carrot", "Cauliflower-cases/ac":"Cauliflower",
                                    "Table Potato-lbs":"Potato", "Seed Potato-lbs":"Potato", "Commercial Pumpkins-Bins/ac":"Pumpkin", "Strawberry Upick-lbs":"Strawberry",
                                    "Pumpkin Upick-pieces/ac":"Pumpkin", "Corn Maze-lbs":"Prairie Pathways", "WW": "Winter Wheat", "Corn (Sweet) Cobs":"Sweet Corn"}
        az_rename_commodity = {"Broccoli-cases/ac":"Broccoli", "Cabbage-lbs/ac":"Cabbage", "Pumpkins-Bins/ac":"Pumpkin", "WatermelonLG-bins/ac": "Watermelon",
                            "WatermelonMini-cases/ac": "Watermelon"}
        bc_rename_commodity = {"Broccoli-cases/ac":"Broccoli", "WatermelonLG-bins/ac": "Watermelon", "WatermelonMini-cases/ac": "Watermelon", "Pumpkins-Bins/ac":"Pumpkin",
                            "Squash-lbs": "Squash"}
        outlook_budget["CommodityRaw"] = outlook_budget["Commodity"]
        production_budget["Commodity"] = production_budget["Commodity"].replace(production_rename_commodity)
        input_budget["Commodity"] = input_budget["Commodity"].replace(input_rename_commodity)
        outlook_budget["Commodity"] = outlook_budget["Commodity"].replace(outlook_rename_commodity)
        az_budget["Commodity"] = az_budget["CommodityRaw"].replace(az_rename_commodity)
        bc_budget["Commodity"] = bc_budget["CommodityRaw"].replace(bc_rename_commodity)
        ## standardizing location naming - merge calderbank grain with Swift Current
        jdlease_rename_location = {"Swift Current Total":"Swift Current", "Regina Farm":"Regina", "Calderbank":"Swift Current",
                                "Airdrie":"Airdrie (grain)", "Eddystone":"Eddystone (grain)"}
        labour_rename_location = {"NexGen (avg met. ton)":"NexGen", "Cache/Fisher/Look":"Aschroft", "MF AZ":"Arizona (produce)", "Box Elder":"Havre", 
                                "BC Veg":"BritishColumbia (produce)","Monette Seeds CDN (avg met. ton)":"Monette Seeds", 
                                "BC Cattle (avg head)":"BritishColumbia (cattle)", "Eddystone Cattle (avg head)":"Eddystone (cattle)",
                                "Swift Current Cattle (avg head)":"Waldeck", "Aridrie Cattle (avg head)":"Airdrie (cattle)",
                                "Airdrie Farm":"Airdrie (grain)", "Eddystone Farm":"Eddystone (grain)","Calderbank":"Calderbank (cattle)"}
        input_rename_location =  {"Fly Creek/Camp 1":"Fly Creek", "Regina Farm":"Regina","Swift Current Total":"Swift Current", "Box Elder":"Havre", "Regina Farm":"Regina",
                                "Calderbank":"Calderbank (grain)","Airdrie":"Airdrie (grain)", "Eddystone":"Eddystone (grain)"}
        production_rename_location = {"Fly Creek/Camp 1":"Fly Creek", "Regina Farm":"Regina","Swift Current Total":"Swift Current", "Box Elder":"Havre", "Regina Farm":"Regina",
                                    "Colorado (Genoa)":"Colorado", "Calderbank":"Swift Current","Airdrie":"Airdrie (grain)", "Eddystone":"Eddystone (grain)"}
        input_budget["Location"] = input_budget["Location"].replace(input_rename_location)
        production_budget["Location"] = production_budget["Location"].replace(production_rename_location)
        labour_budget["Location"] = labour_budget["Location"].replace(labour_rename_location)
        jdlease["Location"] = jdlease["Location"].replace(jdlease_rename_location)
        ## put input budget (chem/fert/seed) into aggregated totals
        input_budget2 = input_budget.groupby(["Location","Type"]).agg({"Amount":"sum"}).reset_index(drop=False)
        ## aggregated totals for production budget and JD Lease
        production_budget = pd.DataFrame(production_budget.groupby(["Location","Currency","Commodity","ForecastPrice"]).agg({"TotalYield":"sum", "ForecastProductionCAD":"sum", "ForecastProductionAdj":"sum"}).reset_index(drop=False))
        jdlease = pd.DataFrame(jdlease.groupby(["Location","Country","Currency","TotalCost25"]).agg({"Acres25":"sum","AllocatedCost25":"sum"}).reset_index(drop=False))
        return input_budget2, production_budget, labour_budget, jdlease, az_budget, bc_budget, outlook_budget

    def _budget_process_produce(self, budget_rules:pd.DataFrame,budget:pd.DataFrame,sheetname:str) -> pd.DataFrame:
        """ 
            this function provides a standardized way to process produce budgets
        """
        budget_rules = budget_rules[budget_rules["SheetRef"] == sheetname].copy(deep=True)
        budget_rules["Commodity"] = budget_rules.apply(lambda x: self._identify_product(x,for_budget=True), axis=1)
        budget["Type"] = budget["Type"].str.strip()
        # gross income - by commodity
        reference = budget[budget["Type"].isin(["Acres","Unit Price","YieldPerAc"])]
        reference = reference.groupby(["Commodity","ProfitType","CommodityRaw"]).agg({"AmountCAD":"prod"}).reset_index(drop=False)
        reference = reference.groupby(["Commodity"]).agg({"AmountCAD":"sum"}).reset_index(drop=False)
        reference = reference.rename(columns={"AmountCAD":"TotalAmountCAD"})
        reference["Category"] = "Produce - production"
        if "outlook" in sheetname.lower():
            for item in ["Prairie Pathways", "Market Garden / CSA"]:
                reference.loc[reference["Commodity"] == item, "Category"] = "Produce - cash settlements"
        # seed expense - by commodity
        expense = budget[budget["Type"] == "Seed"].copy(deep=True)
        expense = expense.drop(columns="CommodityRaw")
        expense = expense.groupby(["Commodity"]).agg({"AmountCAD":"sum"}).reset_index(drop=False).rename(columns={"AmountCAD":"TotalAmountCAD"})
        expense["Category"] = "Seed"
        # other expense - Fertilizer/Chemical - not by commodity
        expense2 = budget[budget["Type"].isin(["Fertilizer","Chemical"])]
        expense2 = expense2.groupby(["Type"]).agg({"AmountCAD":"sum"}).reset_index(drop=False).rename(columns={"AmountCAD":"TotalAmountCAD"})
        expense2["Commodity"] = "Others"
        expense2 = expense2.rename(columns={"Type":"Category"})
        # combine
        budget_produce = pd.merge(budget_rules, pd.concat([reference,expense, expense2]), on=["Commodity","Category"], how="left")
        budget_produce = budget_produce.fillna(value={"TotalAmountCAD":0})
        budget_produce["AmountCAD"] = budget_produce.apply(lambda x: eval(f"{x["TotalAmountCAD"]}{x["Formula"]}"),axis=1)
        return budget_produce

    def _budget_get_transactions(self) -> pd.DataFrame:
        """ 
            get actuals
        """
        if self.is_dev:
            transactions = pd.read_csv(self.gold_path["finance_operational"]/"PL.csv")
            self.operation_acc = pd.read_csv(self.gold_path["finance_operational"]/"AccNumTOAccID.csv")
        else:
            transactions = self.gold_pl.copy(deep=True)
        transactions = transactions[transactions["FiscalYear"] >= 2024]
        transactions["AccName"] = transactions["AccName"].str.strip()
        return transactions

    def _create_budget(self, process_input:bool = False) -> None:
        """ 
            In Progress: this function generates budgets
        """
        print("\nCreating Budget\n")
        if not self.is_dev:
            if not self.pl_exist: self._finance_operational()
        inputdata_path = self.gold_path["budget"] / "Outside Data"
        processed_path = self.gold_path["budget"] / "Processed Data"
        rule_path = self.gold_path["budget"] / "Budget Rules"
        copied_path = self.gold_path["budget"]/"Copied Data"

        # load actuals
        transactions = self._budget_get_transactions()
        
        # process outside data
        if process_input:
            self._buget_process_input(inputdata_path=inputdata_path, processed_path=processed_path)
        
        # read outside data
        input_budget2, production_budget, labour_budget, jdlease, az_budget, bc_budget, outlook_budget = self._budget_read_outsidedata(processed_path=processed_path)

        # calculate Budgets
        ## outside data
        ### read rules
        budget_rules = pd.read_csv(rule_path/"OutsideData.csv")
        budget_rename_category = {"Seed - farm":"Seed"}
        budget_rules["Category"] = budget_rules["Category"].replace(budget_rename_category)
        ### separate locations into individual rows when they are separated with + in the rules df
        budget_rules["Location"] = budget_rules["Location"].str.split("+")
        budget_rules = budget_rules.explode("Location").reset_index(drop=True)
        ### extract formula
        budget_rules = budget_rules.melt(
            id_vars=["Location","Category","AccFull","SheetRef"],
            var_name="Month",
            value_name="Formula"
        )
        budget_rules = budget_rules.fillna(value={"Formula":"0"})
        budget_rules["Formula"] = budget_rules["Formula"].astype(str)
        budget_rules["Formula"] = budget_rules["Formula"].replace({"0": "*0"})
        ### calculating input budget for accounts per location
        budget_rules_input = budget_rules[budget_rules["SheetRef"] == "Input Budget"].copy(deep=True)
        #### workaround input budget for Airdrie grain 
        input_budget2.loc[((input_budget2["Location"]=="Airdrie (grain)")&(input_budget2["Type"]=="Acres")),"Type"] = "Custom work"
        ### merge budget rules with budget total per location
        budget_input = pd.merge(budget_rules_input,input_budget2.rename(columns={"Type":"Category","Amount":"TotalAmountCAD"}),on=["Location","Category"],how="left")
        #### revert back from workaround
        input_budget2.loc[((input_budget2["Location"]=="Airdrie (grain)")&(input_budget2["Type"]=="Custom work")),"Type"] = "Acres"
        ### apply the formula to compute per month
        budget_input["AmountCAD"] = budget_input.apply(lambda x: eval(f"{x["TotalAmountCAD"]}{x["Formula"]}"), axis=1)

        ## production budget
        ### combine Hay and Silage
        production_budget["Commodity"] = production_budget["Commodity"].replace({"Hay":"Hay/Silage", "Silage":"Hay/Silage"})
        ### add commodity column to budget rules
        budget_rules_production = budget_rules[budget_rules["SheetRef"] == "Production Budget"].copy(deep=True)
        budget_rules_production["Commodity"] = budget_rules_production.apply(lambda x: self._identify_product(x, for_budget=True), axis=1)
        ### merge budget rules with budget totals
        budget_production = pd.merge(budget_rules_production,production_budget.loc[:,["Location","Commodity","ForecastProductionCAD"]].rename(columns={"ForecastProductionCAD":"TotalAmountCAD"}),
                                         on = ["Location", "Commodity"], how="left")
        budget_production = budget_production.fillna(value={"TotalAmountCAD":0})
        ### compute budget
        budget_production["AmountCAD"] = budget_production.apply(lambda x: eval(f"{x["TotalAmountCAD"]}{x["Formula"]}"),axis=1)

        ## labour budget
        budget_rules_labour = budget_rules[budget_rules["SheetRef"] == "Labour Budget"].copy(deep=True)
        budget_labour = pd.merge(budget_rules_labour, labour_budget.loc[:,["Location","Month","LabourBudgetCAD"]].rename(columns={"LabourBudgetCAD":"AmountCAD"}),
                                    on=["Location","Month"],how="left")
        
        ## produce budgets
        ### BC
        budget_bc_produce = self._budget_process_produce(budget_rules=budget_rules,budget=bc_budget,sheetname="BC Produce Details")
        ### AZ
        budget_az_produce = self._budget_process_produce(budget_rules=budget_rules,budget=az_budget,sheetname="AZ Details")
        ### outlook
        budget_outlook = self._budget_process_produce(budget_rules=budget_rules,budget=outlook_budget.rename(columns={"Amount":"AmountCAD"}),sheetname="Outlook Details")

        ## JD lease
        budget_rules_jd = budget_rules[budget_rules["SheetRef"]=="JD Lease"].copy(deep=True)
        budget_equipment = pd.merge(budget_rules_jd, jdlease.loc[:,["Location","AllocatedCost25"]].rename(columns={"AllocatedCost25":"TotalAmountCAD"}),
                                        on = "Location", how = "left")
        budget_equipment = budget_equipment.fillna(value={"TotalAmountCAD":0})
        budget_equipment["AmountCAD"] = budget_equipment.apply(lambda x: eval(f"{x["TotalAmountCAD"]}{x["Formula"]}"),axis=1)

        ## adjustment for Swift Current
        months = ["April", "July"]
        for month in months:
            budget_input.loc[((budget_input["Location"]=="Swift Current")&(budget_input["Category"]=="Fertilizer")&(budget_input["Month"]==month)),"TotalAmountCAD"] += \
            budget_input.loc[((budget_input["Location"]=="Calderbank (grain)")&(budget_input["Category"]=="Fertilizer")&(budget_input["Month"]==month)),"TotalAmountCAD"].item()
            budget_input.loc[((budget_input["Location"]=="Swift Current")&(budget_input["Category"]=="Fertilizer")&(budget_input["Month"]==month)),"AmountCAD"] += \
            budget_input.loc[((budget_input["Location"]=="Calderbank (grain)")&(budget_input["Category"]=="Fertilizer")&(budget_input["Month"]==month)),"AmountCAD"].item()
        months = ["June", "September"]
        for month in months:
            budget_input.loc[((budget_input["Location"]=="Swift Current")&(budget_input["Category"]=="Chemical")&(budget_input["Month"]==month)),"TotalAmountCAD"] += \
            budget_input.loc[((budget_input["Location"]=="Calderbank (grain)")&(budget_input["Category"]=="Chemical")&(budget_input["Month"]==month)),"TotalAmountCAD"].item()
            budget_input.loc[((budget_input["Location"]=="Swift Current")&(budget_input["Category"]=="Chemical")&(budget_input["Month"]==month)),"AmountCAD"] += \
            budget_input.loc[((budget_input["Location"]=="Calderbank (grain)")&(budget_input["Category"]=="Chemical")&(budget_input["Month"]==month)),"AmountCAD"].item()
        months = ["May", "June", "September"]
        for month in months:
            budget_input.loc[((budget_input["Location"]=="Swift Current")&(budget_input["Category"]=="Seed")&(budget_input["Month"]==month)),"TotalAmountCAD"] += \
            budget_input.loc[((budget_input["Location"]=="Calderbank (grain)")&(budget_input["Category"]=="Seed")&(budget_input["Month"]==month)),"TotalAmountCAD"].item()
            budget_input.loc[((budget_input["Location"]=="Swift Current")&(budget_input["Category"]=="Seed")&(budget_input["Month"]==month)),"AmountCAD"] += \
            budget_input.loc[((budget_input["Location"]=="Calderbank (grain)")&(budget_input["Category"]=="Seed")&(budget_input["Month"]==month)),"AmountCAD"].item()
        
        # arithmetic rules
        arithmetic = pd.read_csv(rule_path/"Arithmetic.csv")
        ## faltten location
        arithmetic["Location"] = arithmetic["Location"].str.split("+")
        arithmetic = arithmetic.explode("Location").reset_index(drop=True)
        arithmetic_rules = arithmetic.melt(
            id_vars=["Location","Category","AccFull", "AccRef", "FixedRef"],
            var_name="Month",
            value_name="FormulaFull"
        )
        ## housekeeping
        arithmetic_rules = arithmetic_rules.fillna(value={"FormulaFull":"FY-1*0"})
        arithmetic_rules["FormulaFull"] = arithmetic_rules["FormulaFull"].astype(str)
        arithmetic_rules["FormulaFull"] = arithmetic_rules["FormulaFull"].replace({"0":"FY-1*0"})
        arithmetic_rules["ReferenceYear"] = arithmetic_rules["FormulaFull"].str.slice(0,4)
        arithmetic_rules["Formula"] = arithmetic_rules["FormulaFull"].str.slice(4)
        arithmetic_rules = arithmetic_rules.fillna(value={"Formula": "0"})
        arithmetic_rules["Formula"] = arithmetic_rules["Formula"].astype(str)
        arithmetic_rules["Formula"] = arithmetic_rules["Formula"].replace({"0":"*0"})
        ## separating Fixed records
        arithmetic_rules_fixed = arithmetic_rules[arithmetic_rules["AccRef"] == "Fixed"].copy(deep=True)
        arithmetic_rules = arithmetic_rules[arithmetic_rules["AccRef"]!="Fixed"].copy(deep=True)

        ## process fixed records
        arithmetic_rules_fixed = arithmetic_rules_fixed.drop(columns=["FormulaFull","ReferenceYear"]).rename(columns={"FixedRef":"TotalAmountCAD"})
        arithmetic_rules_fixed["AmountCAD"] = arithmetic_rules_fixed.apply(lambda x: eval(f"{x["TotalAmountCAD"]}{x["Formula"]}"),axis=1)

        ## Extract Account Info
        arithmetic_rules["AccNum"] = arithmetic_rules["AccRef"].apply(lambda x: "".join(x.split(" ")[0:2]))
        arithmetic_rules["AccName"] = arithmetic_rules["AccRef"].apply(lambda x: (" ".join(x.split(" ")[2:]).strip()))
        assert "Fixd" not in arithmetic_rules.ReferenceYear.unique(), "Fixd records incorrectly classified"
        ## separate FY-1 & FY+1
        arithmetic_rules_prior = arithmetic_rules[arithmetic_rules["ReferenceYear"] == "FY-1"].copy(deep=True)
        arithmetic_rules = arithmetic_rules[arithmetic_rules["ReferenceYear"] == "FY+1"].copy(deep=True)

        ## process FY-1 with actuals
        actuals = transactions.groupby(["Location", "AccNum", "AccName", "FiscalYear"]).agg({"AmountDisplay":"sum"}).reset_index(drop=False)
        arithmetic_rules_prior["FiscalYear"] = self.currentFY - 1
        assert len(actuals[actuals.duplicated(subset=["AccNum","FiscalYear","Location"],keep=False)]) == 0, "Duplicated AccNum detected for FY-1 Actuals"
        budget_prior = pd.merge(arithmetic_rules_prior,actuals.rename(columns={"AmountDisplay":"TotalAmountCAD"}),
                                on = ["Location","AccNum","FiscalYear"], how="left")
        budget_prior = budget_prior.fillna(value={"TotalAmountCAD": 0})
        budget_prior["AmountCAD"] = budget_prior.apply(lambda x: eval(f"{x["TotalAmountCAD"]}{x["Formula"]}"), axis=1)

        ## processing FY+1 with current budget
        ### budget sales that is based on production budget input sheet
        arithmetic_rules_sales = arithmetic_rules[arithmetic_rules["Category"].str.contains("cash settlements")].copy(deep=True)
        production_reference = pd.concat([budget_production.copy(deep=True), budget_outlook.copy(deep=True), budget_az_produce.copy(deep=True),budget_bc_produce.copy(deep=True)])
        production_reference = production_reference.groupby(["Location","AccFull"]).agg({"AmountCAD":"sum"}).reset_index(drop=False)
        budget_sales = pd.merge(arithmetic_rules_sales,production_reference.rename(columns={"AccFull":"AccRef"}), on=["Location","AccRef"], how="left")
        budget_sales = budget_sales.rename(columns={"AmountCAD":"TotalAmountCAD"})
        budget_sales["AmountCAD"] = budget_sales.apply(lambda x: eval(f"{x["TotalAmountCAD"]}{x["Formula"]}"),axis=1)
        budget_prior = pd.concat([budget_prior, budget_sales],ignore_index=True)

        ### budget inventory adjustment 
        arithmetic_rules_inventory = arithmetic_rules[arithmetic_rules["Category"].str.contains("inventory adjustment",case=False)].copy(deep=True)
        budget_inventory = pd.merge(arithmetic_rules_inventory, budget_prior.loc[:,["Location","AccFull","Month","AmountCAD"]].rename(columns={"AccFull":"AccRef"}),
                                on = ["Location","AccRef", "Month"], how = "left")
        budget_inventory["AmountCAD"] = -budget_inventory["AmountCAD"]
        budget_prior = pd.concat([budget_prior, budget_inventory], ignore_index=True)

        ## combine with fixed budgets
        budget_prior = pd.concat([budget_prior,arithmetic_rules_fixed],ignore_index=True)

        # copied data
        budget_copy = pd.read_csv(copied_path/"Copied Data.csv")
        budget_copy = budget_copy.melt(
            id_vars=["Location","Category","AccFull"],
            var_name = "Month",
            value_name = "AmountCAD"
        )
        budget_copy = budget_copy.fillna(value={"AmountCAD":0})
        budget_copy["AmountCAD"] = budget_copy["AmountCAD"].astype(float)
        budget_copy["FiscalYear"] = self.currentFY
        budget_copy["AccRef"] = "Copy"
        budget_copy["ReferenceYear"] = "NA"
        budget_copy["Formula"] = "NA"
        budget_copy["TotalAmountCAD"] = budget_copy["AmountCAD"]
        budget_copy.loc[budget_copy["Location"]=="Seeds USA", "AmountCAD"] *= self.fx

        # combining all budgets
        budget_outside = pd.concat([budget_input,budget_production,budget_labour,budget_equipment, budget_outlook, budget_az_produce, budget_bc_produce],ignore_index=True)
        budget_outside = budget_outside.drop(columns=["Commodity"])
        budget_prior = budget_prior.drop(columns=["FormulaFull","AccNum","AccName_x", "AccName_y", "AccName", "FixedRef"])
        budget_all = pd.concat([budget_outside,budget_prior,budget_copy],ignore_index=True)
        budget_all["AccNum"] = budget_all["AccFull"].apply(lambda x: "".join(x.split(" ")[0:2]))
        budget_all["AccName"] = budget_all["AccFull"].apply(lambda x: " ".join(x.split(" ")[2:]))
        budget_all["FiscalYear"] = self.currentFY 
        budget_all["DataType"] = "Budget"
        # budget_all.loc[budget_all["Category"].str.contains("inventory adjustment",case=False), "AmountCAD"] *= -1 # turn the sign positive for inventory adjustments (my classification only)

        # save
        self.check_file(self.gold_path["budget"]/"OutputFile")
        budget_all.to_csv(self.gold_path["budget"]/"OutputFile"/"budget_all.csv", index=False)

    def _budget_update(self, force_create:bool=True, force_process_input:bool=False) -> None:
        """ 
            generate/update the actuals from the budget system
        """
        print("\nGenerating/Updating Actuals for budget system\n")
        if self.is_dev:
            self.operation_acc = pd.read_csv(self.gold_path["finance_operational"]/"AccNumTOAccID.csv")
            self.gold_pl = pd.read_csv(self.gold_path["finance_operational"]/"PL.csv")
            self.fx = 1.3807
        else:
            if not self.pl_exist:
                self._finance_operational()
        budget_path = self.gold_path["budget"]/"OutputFile"/"budget_all.csv"
        if (not Path.exists(budget_path)) or force_create:
            self._create_budget(process_input=force_process_input)
        budget = pd.read_csv(budget_path)
        budget = budget.loc[:,["Location", "SheetRef", "Month", "Formula", "TotalAmountCAD", "AmountCAD", "AccRef", "ReferenceYear","FiscalYear", "AccNum", "DataType", "Category"]]
        budget_location_rename = {"Airdrie (grain)": "Airdrie", "Airdrie (cattle)": "Airdrie", "Calderbank (cattle)": "Calderbank",
                                  "Airdrie (corporate)": "Airdrie", "Seeds USA":"Arizona (produce)"}
        budget["Location"] = budget["Location"].replace(budget_location_rename)
        # category_mapping = budget.loc[:,["AccNum", "Category"]].drop_duplicates()     # problem with old changed AccNum mapped to incorrect Category
        # organize Actuals
        transactions = self._budget_get_transactions()
        actuals_all = transactions.groupby(["Location","AccNum", "FiscalYear", "Month"]).agg({"AmountDisplay":"sum"}).reset_index(drop=False)
        actuals_all = actuals_all[actuals_all["FiscalYear"] == self.currentFY]
        actuals_all["DataType"] = "Actual"
        actuals_all = actuals_all.rename(columns={"AmountDisplay": "AmountCAD"})
        # actuals_all = pd.merge(actuals_all,category_mapping,on="AccNum",how="left")   # problem with old changed AccNum mapped to incorrect Category
        actuals_all.to_csv(self.gold_path["budget"]/"OutputFile"/"actuals_all.csv", index=False)
        print(f"Location Unaccounted for in budget: {(set(budget.Location.unique()) - set(actuals_all.Location.unique()))}")
        # combine everything
        all_all = pd.concat([budget,actuals_all],ignore_index=True)
        all_all["FXRate"] = self.fx
        # reroute accounts for changing acc numbers
        all_all["AccNum"] = all_all["AccNum"].replace(self.accnum_reroute)
        # operational classification - AccNum to AccID mapping processed from _finance_operational() function
        assert len(self.operation_acc[self.operation_acc.duplicated(subset=["AccNum"],keep=False)]) == 0, "Duplicated AccNum Detected - Operational Accounts Classification"
        self._extract_accnum_accid()
        all_all["AccID"] = all_all["AccNum"].map(self.acc_map)
        mismatch = all_all[all_all["AccID"].isna()]
        mismatch = mismatch[mismatch['AmountCAD']!=0]
        print(f"Total amount unaccounted for because of accnum mismatching - ${mismatch.AmountCAD.sum()}")
        print(f"AccIDs with non-zero amount: {mismatch.AccNum.unique()}")
        # classify pillars
        all_all["Pillar"] = all_all.apply(lambda x: self._pillar_classification(x), axis=1)
        # save
        all_all.to_csv(self.gold_path["budget"]/"OutputFile"/"all_all.csv", index=False)
        self.check_file(self.gold_path["budget"]/"OutputPowerBI")
        if not self.is_dev: 
            print("Saving ...")
            all_all.to_excel(self.gold_path["budget"]/"OutputPowerBI"/"BudgetActual.xlsx", sheet_name="Budget", index=False)
            for pillar in ["Grain", "Cattle", "Seed", "Produce"]:
                all_all[all_all["Pillar"]==pillar].to_excel(self.gold_path["pillar_dashboard"]/pillar/"BudgetActual.xlsx", sheet_name="Budget", index=False)
            
    def _create_additional_financial(self, summary:pd.DataFrame) -> pd.DataFrame:
        """ 
            this function create Gross Margin, Contribution Margin, EBITDA, Net Income financial terms in the summary table
                summary table must be broken down to fiscal year, month, location
        """
        complement_data = summary.head(0).copy(deep=True)
        for y in summary.FiscalYear.unique():
            subset_year = summary[summary["FiscalYear"] == y]
            for m in summary.Month.unique():
                subset_month = subset_year[subset_year["Month"] == m]
                for l in summary.Location.unique():
                    subset_location = subset_month[subset_month["Location"] == l]
                    if len(subset_location) == 0:
                        continue
                    items = ["Sales Revenue", "Cost of Goods Sold", "Direct Operating Expenses", "Other Operating Revenue", "Operating Overheads", "Other Income", "Other Expense"]
                    values = dict.fromkeys(items, 0)
                    for i in items:
                        if i in subset_location.ProfitType.unique():
                            values[i] = subset_location.loc[subset_location["ProfitType"]==i, "AmountDisplay"].item()
                    gross_margin = values["Sales Revenue"] - values["Cost of Goods Sold"]
                    contribution_margin = gross_margin - values["Direct Operating Expenses"] + values["Other Operating Revenue"]
                    ebitda = contribution_margin - values["Operating Overheads"]
                    net_income = ebitda + values["Other Income"] - values["Other Expense"]
                    pillar = subset_location.Pillar.unique().item()
                    row = {"FiscalYear": y, "Month": m, "Location": l, "Pillar":pillar}
                    row_GM = row | {"ProfitType": "Gross Margin", "AmountDisplay": gross_margin}
                    row_CM = row | {"ProfitType": "Contribution Margin", "AmountDisplay": contribution_margin}
                    row_ebitda = row | {"ProfitType": "EBITDA", "AmountDisplay": ebitda}
                    row_NI = row | {"ProfitType": "Net Income", "AmountDisplay": net_income}
                    complement_data.loc[len(complement_data)] = row_GM
                    complement_data.loc[len(complement_data)] = row_CM 
                    complement_data.loc[len(complement_data)] = row_ebitda 
                    complement_data.loc[len(complement_data)] = row_NI 
        return complement_data

    def _finance_summary(self, create_allocation_reference = True) -> None:
        """ 
            this function assemble summary tables for financial income statement style, including Gross Margin, EBITDA, Net Income, 
                compared to (with % change compared to last year and budget)
                    1. Last Year
                    2. Budget
                    3. month-by-month
                includes Units (e.g., Acres)
        """
        if self.is_dev:
            self.operation_acc = pd.read_csv(self.gold_path["finance_operational"]/"AccNumTOAccID.csv")
            self.gold_pl = pd.read_csv(self.gold_path["finance_operational"]/"PL.csv")
            self.fx = 1.3844
        else:
            if not self.pl_exist:
                self._finance_operational()
        # prepare dfs 
        data = self.gold_pl[self.gold_pl["FiscalYear"] >=  2024].copy(deep=True)
        account = self.gold_acc[self.gold_acc["AccountingType"] == "Income Statement"]
        # create AccID -> ProfitType mapping
        id_prof_map = account.set_index("AccID")["ProfitType"]
        data["ProfitType"] = data["AccID"].map(id_prof_map)
        # summary by location, by pillar, by ProfitType, by Fiscal Year, by Month
        summary = data.groupby(["FiscalYear", "Month", "Location", "Pillar", "ProfitType"]).agg({"AmountDisplay":"sum"}).reset_index(drop=False)
        assert len(summary) == len(data.groupby(["FiscalYear", "Month", "Location", "ProfitType"]).agg({"AmountDisplay":"sum"})), "duplicated location-pillar detected"
        # prepare df for additional financial lines, e.g., Gross Margin, Net Income, ...
        complement_data = self._create_additional_financial(summary)
        # concat two dfs
        summary = pd.concat([summary, complement_data],ignore_index=True)
        # assign datatype before budget summary
        summary["DataType"] = "Actual"
        # read processed budget transactions
        budget = pd.read_csv(self.gold_path["budget"]/"OutputFile"/"all_all.csv")
        budget = budget[budget["DataType"] == "Budget"]
        budget = budget.loc[:,["Location", "Month", "FiscalYear", "AmountCAD", "DataType", "AccID", "Pillar","AccNum"]]
        budget = budget.rename(columns={"AmountCAD":"AmountDisplay"})
        budget = budget[~((budget["AccID"].isna())&(budget["AmountDisplay"] == 0))].reset_index(drop=True)
        assert len(budget[budget["AccID"].isna()]) == 0, f"Unaccounted accounts - {budget[budget["AccID"].isna()].AccNum.unique()}"
        budget = budget.drop(columns=["AccNum"])
        # map ProfitType
        budget["ProfitType"] = budget["AccID"].map(id_prof_map)
        # budget summary
        summary_budget = budget.groupby(["FiscalYear","Month","Location","Pillar","ProfitType"]).agg({"AmountDisplay":"sum"}).reset_index(drop=False)
        assert len(summary_budget) == len(budget.groupby(["FiscalYear","Month","Location","ProfitType"]).agg({"AmountDisplay":"sum"})), "repeat Pillar detected when summarizing budget"
        # additional financial terms
        complement_data_budget = self._create_additional_financial(summary_budget)
        # final processing
        summary_budget = pd.concat([summary_budget,complement_data_budget],ignore_index=True)
        summary_budget["DataType"] = "Budget"
        print(f"Location missing from budget - {set(summary.Location.unique()) - set(summary_budget.Location.unique())}")
        print(f"Location missing from actual - {set(summary_budget.Location.unique()) - set(summary.Location.unique())}")
        # save
        summary_all = pd.concat([summary, summary_budget],ignore_index=True)
        summary_all.to_csv(self.gold_path["finance_operational"]/"ProfitTypeSummary.csv", index=False) # for reclassifying accounts
        summary_all.to_excel(self.gold_path["finance_operational"]/"ProfitTypeSummary.xlsx", sheet_name="ProfitTypeSummary", index=False)
        for pillar in ["Grain", "Cattle", "Seed", "Produce"]:
            summary_all[summary_all["Pillar"]==pillar].to_excel(self.gold_path["pillar_dashboard"]/pillar/"ProfitTypeSummary.xlsx", sheet_name="ProfitTypeSummary", index=False)


    def run(self, force_run_time:bool=False, force_create_budget:bool=True, force_process_budget_input:bool=False) -> None:
        start = perf_counter()

        self._process_units()
        self._weekly_banking()
        self._finance_operational()
        self._payroll_project()
        self._budget_update(force_create=force_create_budget, force_process_input=force_process_budget_input)
        if force_run_time or (self.today.weekday() in [0, 2, 6]): self._QBOTime_project()
        self._hr_summary()
        self._raw_inventory()
        self._finance_summary()

        end = perf_counter()
        print(f"\nProjects Transformation Finished with {(end-start)/60:.3f} minutes\n")


In [39]:
project = Projects()
# project._finance_operational()
project.run(force_run_time=False)


Starting Weekly Banking Project Transformation

None Match Transaction Types
TxnType
Sales Tax Payment    15
Journal Entry         8
BillPaymentCheck      2
Name: count, dtype: int64
Non matches - 25

Starting Finance Operational Project Transformation

location unaccounted for - ['Missing', 'North Farm (deleted)', 'Cache/Fischer/Loon - DNU']
Revising Signs ...

Manual GL Inventory Accounts Adjustments created 48 entries

Saving ...

Starting Payroll Project Transformation

Allocating PPNum for transactions ...
Summarizing ...
Unaccounted location for Acres Doc: {'Airdrie (grain)', 'Seeds USA', 'Home Ranch', 'Colorado', 'Montana (produce)', 'Moon Ranch', 'Fraser River Ranch', 'Diamond S', 'Eddystone (corporate)', 'Ashcroft', 'BritishColumbia (corporate)', 'Wolf Ranch', 'Yorkton'}
Unaccounted location for QBO Payroll: {'Unassigned'}
Saving ...

Generating/Updating Actuals for budget system


Creating Budget

Location Unaccounted for in budget: {'Calderbank (grain)'}
Total amount unacco

In [41]:
"A".lower()

'a'

## APReporting Project

In [8]:
self = Projects()

In [9]:
self.gold_path["APReporting"]

WindowsPath('c:/Users/ZheRao/OneDrive - Monette Farms/Monette Farms Team Site - Innovation Projects/Production/Database/Gold/FinanceProject/APReporting')

In [10]:
os.listdir(self.silver_path["QBO"]["APAR"] / "AgedPayableDetail")

['2025']

In [11]:
def _APAR_concat_memo(self, df:pd.DataFrame) -> pd.DataFrame:
    """ 
        This function concatenates all 'TransactionEntered' column per TransactionID_partial
    """
    df_map = df.loc[:,["TransactionID_partial", "TransactionEntered", "Line_Id", "PrivateNote", "TransactionType"]]
    # fill missing TransactionEntered as Missing so it is string and can be concatenated
    df_map = df_map.fillna(value={"TransactionEntered":"Missing", "PrivateNote": "Missing"})
    # concatenate the line number into TransactionEntered so it's not too messy when concatenate TransactionEntered for multiple lines
    df_map["TransactionEntered"] = df_map["Line_Id"].astype(str) + ". " + df_map["TransactionEntered"]
    # concatenate -> one TransactionEntered per TransactionID_partial
    df_map2 = df_map.sort_values(by=["TransactionID_partial", "Line_Id"],ignore_index=True)\
                    .groupby(["TransactionID_partial","PrivateNote","TransactionType"])["TransactionEntered"].agg(" ".join).reset_index(drop=False)
    assert len(df_map2[df_map2.duplicated(subset="TransactionID_partial")]) == 0, "duplicated transactionID spotted when creating TransactionEntered Concatenation"
    return df_map2

In [12]:
def _APRporting_project(self, date:set[int] = None) -> None:
    """ 
        This function connects APAgingDetails report from QBO API, and combined with raw tables such as Bill to form a comprehensive report that meets finance team's need
            This function supports date input for processing the report at that exact date
    """
    if date is not None:
        assert len(date) == 3, f"please pass the date as (YYYY, M, D), passed {date}"
        year, month, day = date
    else:
        year, month, day = self.today.year, self.today.month, self.today.day
    # read APAging report from silver space
    try:
        report = pd.read_csv(self.silver_path["QBO"]["APAR"] / "AgedPayableDetail" / str(year) / str(month) / f"{day}.csv")
    except:
        print(f'csv file not found at {self.silver_path["QBO"]["APAR"] / "AgedPayableDetail" / str(year) / str(month) / f"{day}.csv"}')
    report = report.rename(columns={"TransactionTypeID":"TransactionID_partial"})
    report_transactiontype_rename = {"Bill Payment (Cheque)":"BillPaymentCheck", "Bill Payment (Credit Card)":"BillPaymentCheck", "Bill Payment (Check)":"BillPaymentCheck",
                                     "Cheque Expense":"Purchase", "Supplier Credit": "Vendor Credit"}
    # working on Bill, VendorCredit, and Journal Entry for now
    report["TransactionType"] = report["TransactionType"].replace(report_transactiontype_rename)
    # report = report[report["TransactionType"].isin(["Bill", "Vendor Credit", "Journal Entry"])]
    return report

In [13]:
report =_APRporting_project(self)
report.TransactionType.value_counts()

TransactionType
Bill                2278
Vendor Credit        191
Journal Entry         54
BillPaymentCheck       9
Purchase               1
Name: count, dtype: int64

In [14]:
len(report[report["TransactionType"]=="Journal Entry"].TransactionID_partial.unique())

52

In [15]:
os.listdir(self.silver_path["QBO"]["Raw"])

['Bill.csv',
 'CreditMemo.csv',
 'Deposit.csv',
 'Invoice.csv',
 'JournalEntry.csv',
 'LinkedTxn',
 'Purchase.csv',
 'SalesReceipt.csv',
 'VendorCredit.csv']

In [16]:
cols = ["TransactionDate", "TotalAmt", "PrivateNote", "APAccID", "DocNumber", "TransactionEntered", "Amount", "TransactionID", "VendorID", "FarmID", "TransactionID_partial", "Line_Id",
        "AccID"]
bill_col = cols + ["TermID"]
bill = pd.read_csv(self.silver_path["QBO"]["Raw"]/"Bill.csv",usecols=bill_col)
print(len(bill))
bill = bill[bill["TransactionID_partial"].isin(report[report["TransactionType"]=="Bill"]["TransactionID_partial"].unique())]
print(len(bill))
bill["TransactionType"] = "Bill"
bill.to_excel(self.gold_path["APReporting"]/"Bill.xlsx", index=False, sheet_name="Bill")
bill.head()

74798
2995


,TransactionID_partial,TransactionDate,TotalAmt,PrivateNote,TermID,FarmID,VendorID,APAccID,DocNumber,Line_Id,TransactionEntered,Amount,AccID,TransactionID,TransactionType
0,MFUSA16255,2025-09-16,31.25,Approved by David for Corp Sep 29/25,NaN,MFUSA4,MFUSA46,MFUSA65,MFUSA-53780,1,Legal Services for Corporate Matters RE letter...,31.25,MFUSA8,B-MFUSA16255-1,Bill
1,MFUSA16256,2025-09-18,8000.00,Sent to Nat for approval Sep 25/25\n Wire to A...,NaN,MFUSA4,MFUSA44,MFUSA65,MFUSA-Sep 18/25,1,"Administrative Assessment for Oct, Nov, Dec 2025",8000.00,MFUSA8,B-MFUSA16256-1,Bill
2,MFUSA16254,2025-09-26,10.49,NaN,NaN,MFUSA5,MFUSA6,MFUSA65,MFUSA-367496,1,Crescent Tape Measure,10.49,MFUSA172,B-MFUSA16254-1,Bill
3,MFUSA16253,2025-09-27,785.00,Approved by Jake Sep 29/25,NaN,MFUSA5,MFUSA473,MFUSA65,MFUSA-Sep 27/25,1,"Replaced dishwasher and lines, add outlet for ...",785.00,MFUSA56,B-MFUSA16253-1,Bill
4,MFUSA16251,2025-09-26,2528.06,NaN,NaN,MFUSA5,MFUSA6,MFUSA65,MFUSA-449156,1,Dyed Diesel x 857,2528.06,MFUSA155,B-MFUSA16251-1,Bill


In [17]:
bill_map = _APAR_concat_memo(self,bill)
bill_map

,TransactionID_partial,PrivateNote,TransactionType,TransactionEntered
0,MFAZ10726,Sent to Ed for approval Apr 11/25,Bill,1. Repack Cooler
1,MFAZ10727,Sent to Ed/Dan/RM for approval Apr 11/25,Bill,1. Workers on Green Cabbage
2,MFAZ10843,Sent to Ed/Dan/RM for approval Apr 28/25,Bill,1. Custom Hauling
3,MFAZ12177,Approved by Nat Aug 27/25,Bill,"1. KE-Lease No 003-078735-00 for November 7th,..."
4,MFAZ12178,Matches MO2506165 but MO not yet approved by D...,Bill,1. Ele-Max Soil Phosphite
...,...,...,...,...
2273,NexGen6214,Parts for repairs to plant. Did not pay for th...,Bill,1. Parts for repairs to plant. Did not pay for...
2274,NexGen6396,Approved by Christiaan Jul 22/25\nInvoice rece...,Bill,"1. December 16/17, 2024 Sales Training - Justi..."
2275,NexGen6558,Approved by Christiaan Sep 5/25,Bill,1. Troubleshoot booster problem. Parts and Labour
2276,NexGen6559,Approved by Jordie Sep 11/25,Bill,1. Re-Inspection Assessment


In [18]:
vc = pd.read_csv(self.silver_path["QBO"]["Raw"] / "VendorCredit.csv", usecols=cols)
print(len(vc))
vc = vc[vc["TransactionID_partial"].isin(report[report["TransactionType"]=="Vendor Credit"]["TransactionID_partial"].unique())]
print(len(vc))
vc["TransactionType"] = "Vendor Credit"
vc.to_excel(self.gold_path["APReporting"]/"VendorCredit.xlsx", index=False, sheet_name="VendorCredit")
vc.head()

3825
268


,TransactionID_partial,DocNumber,TransactionDate,TotalAmt,PrivateNote,FarmID,VendorID,APAccID,Line_Id,TransactionEntered,Amount,AccID,TransactionID,TransactionType
0,MFUSA15434,MFUSA-FINCHRG ADJ,2025-07-21,26391.59,NaN,MFUSA5,MFUSA9,MFUSA65,1,Finance Charge Adjustment,26391.59,MFUSA142,V-MFUSA15434-1,Vendor Credit
1,MFUSA15431,MFUSA-57563453,2025-07-09,22075.85,NaN,MFUSA5,MFUSA9,MFUSA65,1,RETURN: PRIAXOR XEMIUM 2X2.5GA,14325.00,MFUSA107,V-MFUSA15431-1,Vendor Credit
2,MFUSA15431,MFUSA-57563453,2025-07-09,22075.85,NaN,MFUSA5,MFUSA9,MFUSA65,2,RETURN: RIFLE HERBICIDE 2X2.5GA,66.25,MFUSA107,V-MFUSA15431-2,Vendor Credit
3,MFUSA15431,MFUSA-57563453,2025-07-09,22075.85,NaN,MFUSA5,MFUSA9,MFUSA65,3,RETURN:REVYLOK 2X2.5GA,5103.35,MFUSA107,V-MFUSA15431-3,Vendor Credit
4,MFUSA15431,MFUSA-57563453,2025-07-09,22075.85,NaN,MFUSA5,MFUSA9,MFUSA65,4,RETURN:SPREADER 90 2X2.5GA,2581.25,MFUSA107,V-MFUSA15431-4,Vendor Credit


In [19]:
vc_map = _APAR_concat_memo(self, vc)
vc_map

,TransactionID_partial,PrivateNote,TransactionType,TransactionEntered
0,MFAZ12352,Missing,Vendor Credit,1. CAB Supreme Vantage for invoice INV56855
1,MFBC13652,Missing,Vendor Credit,1. Missing
2,MFBC22539,Ed approved Oct 17,Vendor Credit,1. Return livestock watering thermostat
3,MFBC22946,Matt approval requested Nov 5th (all Fischer r...,Vendor Credit,1. Fischer Ranch replacement pivots
4,MFBC23078,Credit issued per TZ with communication with SI,Vendor Credit,1. Credit per TZ communication with Walter at SI
...,...,...,...,...
186,NexGen6404,Missing,Vendor Credit,1. $100 DEPOSIT 2. RAXIL PRO 58.5
187,NexGen6457,Credit sent to Jordie for review Aug 8/25,Vendor Credit,1. Branch and CSGA Acreage fees
188,NexGen6599,Interest adjustment approved by Brook 081225,Vendor Credit,1. Interest adjustment approved by Brook
189,NexGen6600,Missing,Vendor Credit,1. Refund of Finance Charges due Sep/25


In [20]:
journal_cols = cols + ["JEType"]
journal = pd.read_csv(self.silver_path["QBO"]["Raw"] / "JournalEntry.csv", usecols=[x for x in journal_cols if x not in ['TotalAmt', 'APAccID']])
print(len(journal))
journal = journal[journal["TransactionID_partial"].isin(report[report["TransactionType"]=="Journal Entry"]["TransactionID_partial"].unique())]
print(len(journal))
journal["TransactionType"] = "Journal Entry"
journal.to_excel(self.gold_path["APReporting"]/"JournalEntry.xlsx", index=False, sheet_name="JournalEntry")
journal.head()

101733
149


,TransactionID_partial,DocNumber,TransactionDate,PrivateNote,Line_Id,TransactionEntered,Amount,JEType,AccID,FarmID,TransactionID,VendorID,TransactionType
34,MFUSA16197,MFUSA-1916,2025-09-23,"Will come back to post payment to invoices, ne...",0,ACH to Nutrien for Aug Stmt Payment,535513.38,Credit,MFUSA32,NaN,J-MFUSA16197-0,NaN,Journal Entry
35,MFUSA16197,MFUSA-1916,2025-09-23,"Will come back to post payment to invoices, ne...",1,ACH to Nutrien for Aug Stmt Payment,535513.38,Debit,MFUSA65,NaN,J-MFUSA16197-1,MFUSA9,Journal Entry
128,MFUSA16122,MFUSA-1904,2025-09-10,NaN,0,Simplot payment from MFL,578447.86,Credit,MFUSA75,NaN,J-MFUSA16122-0,NaN,Journal Entry
129,MFUSA16122,MFUSA-1904,2025-09-10,NaN,1,Simplot payment from MFL,578447.86,Debit,MFUSA65,NaN,J-MFUSA16122-1,MFUSA216,Journal Entry
1107,MFUSA14896,MFUSA-1740,2025-07-02,"Will come back to post payment to invoices, ne...",0,ACH to Nutrien for June Stmt Payment,1194627.78,Credit,MFUSA32,NaN,J-MFUSA14896-0,NaN,Journal Entry


In [21]:
journal_map = _APAR_concat_memo(self,journal)
len(journal_map)

51

In [22]:
fact = pd.concat([bill, vc, journal], ignore_index=True)
fact.to_excel(self.gold_path["APReporting"]/"Facts.xlsx", index=False, sheet_name = "Facts")

In [23]:
fact_map = pd.concat([bill_map, vc_map, journal_map],ignore_index=True)
fact_map

,TransactionID_partial,PrivateNote,TransactionType,TransactionEntered
0,MFAZ10726,Sent to Ed for approval Apr 11/25,Bill,1. Repack Cooler
1,MFAZ10727,Sent to Ed/Dan/RM for approval Apr 11/25,Bill,1. Workers on Green Cabbage
2,MFAZ10843,Sent to Ed/Dan/RM for approval Apr 28/25,Bill,1. Custom Hauling
3,MFAZ12177,Approved by Nat Aug 27/25,Bill,"1. KE-Lease No 003-078735-00 for November 7th,..."
4,MFAZ12178,Matches MO2506165 but MO not yet approved by D...,Bill,1. Ele-Max Soil Phosphite
...,...,...,...,...
2515,MSL10244,Missing,Journal Entry,"0. Pymt to Ray-Mont - $220,972.26USD @ 1.3758F..."
2516,MSL10480,Missing,Journal Entry,0. USD EFT Pymt to Ray-Mont posted as a JE unt...
2517,MSL10735,Missing,Journal Entry,0. Pymt to Ray-Mont posted as a JE until CC ca...
2518,MSL10970,Missing,Journal Entry,0. EFT sent to RayMont Logstics. Requested inf...


In [24]:
report2 = pd.merge(report, fact_map, on=["TransactionID_partial", "TransactionType"], how="left")
report2

,Date,TransactionType,DocNumber,Vendor,Farm,DueDate,PastDue,Amount,OpenBalance,VendorID,TransactionID_partial,FarmID,AmountCAD,APCategory,Corp,Country,FXRate,ReportDate,PrivateNote,TransactionEntered
0,2022-08-01,Bill,MFUSA-17165,Safflower Technologies International,Billings,2022-08-01,1156.0,7308.00,7308.00,MFUSA174,MFUSA2144,MFUSA2,10169.812800,91 or more days past due,MFUSA,USA,1.3916,2025-09-30,Sent to Deon/Phil for Approval,1. Railcar loading fees
1,2022-08-01,Bill,MFUSA-17300,Safflower Technologies International,Billings,2022-08-01,1156.0,4828.34,4828.34,MFUSA174,MFUSA2396,MFUSA2,6719.117944,91 or more days past due,MFUSA,USA,1.3916,2025-09-30,Sent to Deon/Phil for Approval,1. Railcar loading fees
2,2022-08-01,Bill,MFUSA-17195,Safflower Technologies International,Billings,2022-08-01,1156.0,3900.00,3900.00,MFUSA174,MFUSA2142,MFUSA2,5427.240000,91 or more days past due,MFUSA,USA,1.3916,2025-09-30,Sent to Deon/Phil for Approval,1. Railcar loading fees
3,2022-08-01,Bill,MFUSA-17202,Safflower Technologies International,Billings,2022-08-01,1156.0,1080.00,1080.00,MFUSA174,MFUSA2143,MFUSA2,1502.928000,91 or more days past due,MFUSA,USA,1.3916,2025-09-30,Sent to Deon/Phil for Approval,1. Railcar loading fees
4,2023-01-24,Bill,MFUSA-E06899,Redhead Equipment,-Corporate,2023-01-24,980.0,767.60,767.60,MFUSA147,MFUSA3202,MFUSA4,1068.192160,91 or more days past due,MFUSA,USA,1.3916,2025-09-30,Do not pay yet as per Russ Mar 3/23,1. Invoice acct for Torgerson invoice W14795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2528,2025-09-09,Vendor Credit,MFL-500011948204 Sep/25,SaskPower,Outlook,0-00-00,0.0,-19.35,-19.35,MFL22,MFL100659,MFL14,-19.350000,91 or more days past due,MFL,Canada,1.3916,2025-09-30,Missing,1. Jun-Sep Power @ NW3531073
2529,2025-09-12,Vendor Credit,MFL-035-546453,Gregg Distributors (Outlook 249050),Outlook,0-00-00,0.0,-8.10,-8.10,MFL1307,MFL101504,MFL14,-8.100000,91 or more days past due,MFL,Canada,1.3916,2025-09-30,Missing,1. 035-539896 2PK RAT GLUE TRAPS
2530,2025-09-18,Vendor Credit,MFL-10171058,Pioneer Co-op Regina,Regina,0-00-00,0.0,-9350.00,-9350.00,MFL257,MFL101792,MFL3,-9350.000000,91 or more days past due,MFL,Canada,1.3916,2025-09-30,Missing,1. Mondanto Deposit 800L
2531,2025-09-22,Vendor Credit,MFL-437-903461,Napa Auto Parts,Swift Current,0-00-00,0.0,-24.85,-24.85,MFL3805,MFL101530,MFL5,-24.850000,91 or more days past due,MFL,Canada,1.3916,2025-09-30,"Paid by visa 6783 (JP)\nReturns ,credit note","1. Paid by visa 6783 (JP)\nReturns ,credit note"


In [25]:
report2.to_excel(self.gold_path["APReporting"]/"APReport.xlsx", index=False, sheet_name="APReport")

In [26]:
os.listdir()

['AgedReceivableDetail_2025_9_24.json',
 'dev.ipynb',
 'harvestprofit.ipynb',
 'old_dev.ipynb']

In [27]:
vendor = pd.read_csv(self.silver_path["QBO"]["Dimension"]/"CSV"/"Vendor.csv")
vendor.to_excel(self.gold_path["APReporting"].parent / "Vendor.xlsx", sheet_name = "Vendor", index=False)

In [28]:
farm = pd.read_csv(self.silver_path["QBO"]["Dimension"]/"CSV"/"Farm.csv")
farm.to_excel(self.gold_path["APReporting"].parent / "Location.xlsx", sheet_name = "Location", index=False)

# Unclassified - reclassification

In [4]:
self = Projects(is_dev=True)
if self.is_dev:
    accounts = self.silver_acc
    with open(self.silver_path["QBO"]["Dimension"]/"acc_classification.yaml", "r", encoding="utf-8") as f:
        raw_acc = yaml.safe_load(f)
    rows = [(l1, l2, l3, v) 
            for l1, l1_inner in raw_acc.items() 
            for l2, l2_inner in l1_inner.items() 
            for l3, l3_inner in l2_inner.items() 
            for v in l3_inner]
    acc_operation = pd.DataFrame(rows, columns=["OperationProfType", "OperationCategory", "OperationSubCategory", "AccID"])
    accounts = pd.merge(accounts, acc_operation, on = "AccID", how = "left")
    accounts["Commodity"] = accounts.apply(lambda x: self._identify_product(x), axis=1)
    self.gold_acc = accounts
    self.gold_pl = pd.read_csv(self.gold_path["finance_operational"]/"PL.csv")
else:
    if not self.pl_exist:
        self._finance_operational()
len(self.gold_acc), len(self.gold_pl)

(3588, 192181)

In [5]:
data = self.gold_pl[self.gold_pl["FiscalYear"] == self.currentFY].copy(deep=True)
data = data[data["Pillar"].isin(["Unclassified", "Missing"])]
account = self.gold_acc[self.gold_acc["AccountingType"] == "Income Statement"]
len(data), len(account)

(5184, 1937)

In [20]:
data.Location.value_counts()

Location
Corporate                      3868
Missing                         680
BritishColumbia (corporate)     406
Eddystone (corporate)           207
Cache/Fischer/Loon - DNU         23
Name: count, dtype: int64

## compute allocation table

In [10]:
summary = pd.read_csv(self.gold_path["finance_operational"]/"ProfitTypeSummary.csv")
summary = summary[summary["FiscalYear"] == self.currentFY]
summary = summary[((summary["DataType"] == "Budget")&(summary["ProfitType"]=="EBITDA"))]
summary

,FiscalYear,Month,Location,Pillar,ProfitType,AmountDisplay,DataType
8417,2025,April,Airdrie,Cattle,EBITDA,1.042472e+06,Budget
8421,2025,April,Arizona (produce),Produce,EBITDA,8.047312e+06,Budget
8425,2025,April,Ashcroft,Cattle,EBITDA,7.371964e+05,Budget
8429,2025,April,BritishColumbia (corporate),Unclassified,EBITDA,-3.335495e+04,Budget
8433,2025,April,BritishColumbia (produce),Produce,EBITDA,-4.955663e+05,Budget
...,...,...,...,...,...,...,...
9837,2025,September,Swift Current,Grain,EBITDA,1.124738e+07,Budget
9841,2025,September,The Pas,Grain,EBITDA,3.476375e+06,Budget
9845,2025,September,Waldeck,Cattle,EBITDA,1.153897e+06,Budget
9849,2025,September,Wolf Ranch,Cattle,EBITDA,2.262920e+03,Budget


In [11]:
summary = summary.groupby(["Location"]).agg({"AmountDisplay":"sum"}).reset_index(drop=False)
summary = summary.rename(columns={"AmountDisplay":"ProjectedETBIDA"})
summary

,Location,ProjectedETBIDA
0,Airdrie,5.818445e+06
1,Arizona (produce),4.355382e+06
2,Ashcroft,8.650375e+05
3,BritishColumbia (corporate),-7.147001e+05
4,BritishColumbia (produce),1.026197e+06
5,Calderbank,1.393097e+06
6,Calderbank (grain),-6.715892e+05
7,Camp 4,4.917566e+06
8,Diamond S,1.898820e+05
9,Eddystone (cattle),8.803880e+05


In [ ]:
# corporate/missing -> all locations
# BC corporate -> BC ranches
# Eddystone corporate -> Eddystone Grain + Eddystone Cattle
location_exclude_transaction = ['Arizona (grain)', 'Billings', 'Seeds USA']
all_locations = self.locations["Seed"] + self.locations["Grain"] + self.locations["Produce"] + self.locations["Cattle"]
all_locations = list(set(all_locations) - set(location_exclude_transaction))
reallocation = {
    "Corporate": all_locations,
    "Missing": all_locations,
    "BritishColumbia (corporate)": self.bc_ranches,
    "Eddystone (corporate)": ["Eddystone (grain)", "Eddystone (cattle)"]
}

In [13]:
summary = summary[summary["Location"].isin(all_locations)]
summary.Location.unique()

array(['Airdrie', 'Arizona (produce)', 'Ashcroft',
       'BritishColumbia (produce)', 'Calderbank', 'Camp 4', 'Diamond S',
       'Eddystone (cattle)', 'Eddystone (grain)', 'Fly Creek',
       'Fraser River Ranch', 'Hafford', 'Havre', 'Home Ranch', 'Kamsack',
       'Moon Ranch', 'NexGen', 'Outlook', 'Prince Albert', 'Raymore',
       'Regina', 'Seeds', 'Swift Current', 'The Pas', 'Waldeck',
       'Wolf Ranch', 'Yorkton'], dtype=object)

In [14]:
len(summary.Location.unique())

27

In [15]:
# add 0 amount to new locations for allocating 0 total amount
print(len(summary))
new_locations = set(all_locations) - set(summary.Location.unique())
if len(new_locations) > 0:
    new_df = summary.head(0).copy(deep=True)
    for l in new_locations:
        new_df.loc[len(new_df)] = {"Location": l, "ProjectedETBIDA": 0}
    summary = pd.concat([summary, new_df], ignore_index=True)
print(len(summary))

27
29


In [16]:
summary.Location.unique()

array(['Airdrie', 'Arizona (produce)', 'Ashcroft',
       'BritishColumbia (produce)', 'Calderbank', 'Camp 4', 'Diamond S',
       'Eddystone (cattle)', 'Eddystone (grain)', 'Fly Creek',
       'Fraser River Ranch', 'Hafford', 'Havre', 'Home Ranch', 'Kamsack',
       'Moon Ranch', 'NexGen', 'Outlook', 'Prince Albert', 'Raymore',
       'Regina', 'Seeds', 'Swift Current', 'The Pas', 'Waldeck',
       'Wolf Ranch', 'Yorkton', 'Montana (produce)', 'Colorado'],
      dtype=object)

In [17]:
all_perc = pd.DataFrame()
for keys in reallocation.keys():
    from_location = keys
    temp = pd.DataFrame({"from_location": [from_location] * len(reallocation[from_location]), "to_location": reallocation[from_location]})
    temp = pd.merge(temp, summary.rename(columns={"Location": "to_location"}), on="to_location", how="left")
    temp["ProjectedETBIDAAdj"] = temp["ProjectedETBIDA"].clip(lower=0)
    temp["Total"] = temp.ProjectedETBIDAAdj.sum()
    temp["Perc"] = temp["ProjectedETBIDAAdj"] / temp["Total"]
    assert abs(1 - temp.Perc.sum()) < 0.0001, f"Percentages don't add up for {keys}"
    all_perc = pd.concat([all_perc,temp],ignore_index=True)

In [18]:
all_perc

,from_location,to_location,ProjectedETBIDA,ProjectedETBIDAAdj,Total,Perc
0,Corporate,The Pas,2.327334e+06,2.327334e+06,9.434081e+07,0.024669
1,Corporate,Havre,3.640877e+06,3.640877e+06,9.434081e+07,0.038593
2,Corporate,Yorkton,9.150557e+05,9.150557e+05,9.434081e+07,0.009699
3,Corporate,Calderbank,1.393097e+06,1.393097e+06,9.434081e+07,0.014767
4,Corporate,Moon Ranch,5.403023e+05,5.403023e+05,9.434081e+07,0.005727
...,...,...,...,...,...,...
61,BritishColumbia (corporate),Wolf Ranch,5.777612e+05,5.777612e+05,3.344086e+06,0.172771
62,BritishColumbia (corporate),Diamond S,1.898820e+05,1.898820e+05,3.344086e+06,0.056781
63,BritishColumbia (corporate),Home Ranch,7.263841e+05,7.263841e+05,3.344086e+06,0.217215
64,Eddystone (corporate),Eddystone (grain),4.927422e+05,4.927422e+05,1.373130e+06,0.358846


In [19]:
all_perc.to_csv(self.gold_path["finance_operational"]/"allocation_schedule.csv", index=False)

## allocation unclassified transactions

In [21]:
add_all = pd.DataFrame()
for from_location in data.Location.unique():
    if from_location == "Cache/Fischer/Loon - DNU":
        continue
    print(f"processing {from_location}")
    add_df = data.head(0).copy(deep=True)
    # from_location = "Corporate"
    # focus percentage for corporate
    perc_sub = all_perc[all_perc["from_location"] == from_location].copy(deep=True)
    # actual transactions
    transactions = data[data["Location"] == from_location].copy(deep=True)
    for to_location in perc_sub.to_location.unique():
        # focus on percentages to income, and expense for to_location
        perc_sub2 = perc_sub[perc_sub["to_location"]==to_location]
        assert len(perc_sub2) == 1, f"repeated to_location at {to_location} from {from_location} detected"
        perc = perc_sub2.Perc.unique()[0]
        # make a copy of the transaction and apply percentage for alllocation for this particular to_location
        temp = transactions.copy(deep=True)
        for col in [column for column in temp.columns if "Amount" in column]:
            temp[col] *= perc
        # adjust the column values for 
        ##      "Not Applicable" - DocNumber, FarmID, Pillar 
        ##      Memo = "Algorithmic Reallocation"
        ##      Location = to_location
        for col in ["DocNumber", "FarmID", "Pillar"]:
            temp[col] = "Not Applicable"
        temp["Memo"] = "Algorithmic Reallocation"
        temp["Location"] = to_location
        add_df = pd.concat([add_df, temp],ignore_index=True)
    add_all = pd.concat([add_all, add_df],ignore_index=True)
    assert abs(transactions.AmountDisplay.sum() - add_df.AmountDisplay.sum()) < 0.01, "Total amounts of splitting transactions do not match"

processing Corporate
processing Missing
processing BritishColumbia (corporate)
processing Eddystone (corporate)


In [22]:
add_all

,TransactionDate,TransactionType,TransactionID_partial,DocNumber,Name,NameID,LocationRaw,FarmID,Class,Memo,...,ClassID,AmountAdj,AmountCAD,FXRate,Country,FiscalYear,Month,Location,Pillar,AmountDisplay
0,2024-11-01,Journal Entry,MFUSA12431,Not Applicable,NaN,MFUSA,-Corporate,Not Applicable,NaN,Algorithmic Reallocation,...,NaN,-20995.732764,-28936.318895,1.3782,USA,2025,November,The Pas,Not Applicable,28936.318895
1,2024-11-29,Journal Entry,MFUSA12147,Not Applicable,NaN,MFUSA,-Corporate,Not Applicable,NaN,Algorithmic Reallocation,...,NaN,20199.160993,27838.483681,1.3782,USA,2025,November,The Pas,Not Applicable,-27838.483681
2,2024-11-29,Bill,MFUSA12146,Not Applicable,Simplot Grower Solutions Billings,MFUSA216,-Corporate,Not Applicable,NaN,Algorithmic Reallocation,...,NaN,-20199.160993,-27838.483681,1.3782,USA,2025,November,The Pas,Not Applicable,27838.483681
3,2024-11-29,Vendor Credit,MFUSA12148,Not Applicable,Simplot Grower Solutions Billings,MFUSA216,-Corporate,Not Applicable,NaN,Algorithmic Reallocation,...,NaN,20995.732764,28936.318895,1.3782,USA,2025,November,The Pas,Not Applicable,-28936.318895
4,2024-12-31,Journal Entry,MFUSA12752,Not Applicable,NaN,MFUSA,-Corporate,Not Applicable,NaN,Algorithmic Reallocation,...,NaN,-796.571771,-1097.835214,1.3782,USA,2025,December,The Pas,Not Applicable,1097.835214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134737,2025-07-31,Journal Entry,MFL98970,Not Applicable,NaN,MFL,Eddystone - General,Not Applicable,NaN,Algorithmic Reallocation,...,NaN,-2820.577649,-2820.577649,1.3782,Canada,2025,July,Eddystone (cattle),Not Applicable,2820.577649
134738,2025-08-15,Bill,MFL100036,Not Applicable,Rm of Alonsa,MFL3993,Eddystone - General,Not Applicable,Corporate,Algorithmic Reallocation,...,MFL618279,-5523.657409,-5523.657409,1.3782,Canada,2025,August,Eddystone (cattle),Not Applicable,5523.657409
134739,2025-08-15,Bill,MFL100033,Not Applicable,Rm of Alonsa,MFL3993,Eddystone - General,Not Applicable,Corporate,Algorithmic Reallocation,...,MFL618279,-13490.874660,-13490.874660,1.3782,Canada,2025,August,Eddystone (cattle),Not Applicable,13490.874660
134740,2025-08-15,Bill,MFL100035,Not Applicable,Rm of Alonsa,MFL3993,Eddystone - General,Not Applicable,Corporate,Algorithmic Reallocation,...,MFL618279,-24289.614059,-24289.614059,1.3782,Canada,2025,August,Eddystone (cattle),Not Applicable,24289.614059


In [23]:
add_all.AmountDisplay.sum()

73243386.21808001

In [24]:
data[data["Location"]!="Cache/Fischer/Loon - DNU"].AmountDisplay.sum()

73243386.21808

In [25]:
offset = data[data["Location"]!="Cache/Fischer/Loon - DNU"].copy(deep=True)
for col in [column for column in offset.columns if "Amount" in column]:
    offset[col] *= -1
for col in ["FarmID", "Pillar"]:
    offset[col] = "Not Applicable"
offset["Memo"] = "Algorithmic Offset"
offset

,TransactionDate,TransactionType,TransactionID_partial,DocNumber,Name,NameID,LocationRaw,FarmID,Class,Memo,...,ClassID,AmountAdj,AmountCAD,FXRate,Country,FiscalYear,Month,Location,Pillar,AmountDisplay
168,2024-11-01,Journal Entry,MFUSA12431,MFUSA-1440R,NaN,MFUSA,-Corporate,Not Applicable,NaN,Algorithmic Offset,...,NaN,851083.20,1.172963e+06,1.3782,USA,2025,November,Corporate,Not Applicable,-1.172963e+06
173,2024-11-29,Journal Entry,MFUSA12147,MFUSA-1392,NaN,MFUSA,-Corporate,Not Applicable,NaN,Algorithmic Offset,...,NaN,-818793.36,-1.128461e+06,1.3782,USA,2025,November,Corporate,Not Applicable,1.128461e+06
174,2024-11-29,Bill,MFUSA12146,MFUSA-731068069,Simplot Grower Solutions Billings,MFUSA216,-Corporate,Not Applicable,NaN,Algorithmic Offset,...,NaN,818793.36,1.128461e+06,1.3782,USA,2025,November,Corporate,Not Applicable,-1.128461e+06
175,2024-11-29,Vendor Credit,MFUSA12148,MFUSA-731068068,Simplot Grower Solutions Billings,MFUSA216,-Corporate,Not Applicable,NaN,Algorithmic Offset,...,NaN,-851083.20,-1.172963e+06,1.3782,USA,2025,November,Corporate,Not Applicable,1.172963e+06
178,2024-12-31,Journal Entry,MFUSA12752,MFUSA-1474,NaN,MFUSA,-Corporate,Not Applicable,NaN,Algorithmic Offset,...,NaN,32289.84,4.450186e+04,1.3782,USA,2025,December,Corporate,Not Applicable,-4.450186e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54547,2025-07-17,Journal Entry,MFL97048,MFL-7801,NaN,MFL,NaN,Not Applicable,NaN,Algorithmic Offset,...,NaN,5228.38,5.228380e+03,1.3782,Canada,2025,July,Missing,Not Applicable,-5.228380e+03
54548,2025-08-05,Journal Entry,MFL98583,MFL-7901,NaN,MFL,NaN,Not Applicable,NaN,Algorithmic Offset,...,NaN,57101.36,5.710136e+04,1.3782,Canada,2025,August,Missing,Not Applicable,-5.710136e+04
54549,2025-08-05,Journal Entry,MFL98585,MFL-7903,NaN,MFL,NaN,Not Applicable,NaN,Algorithmic Offset,...,NaN,7434.25,7.434250e+03,1.3782,Canada,2025,August,Missing,Not Applicable,-7.434250e+03
54550,2025-08-05,Journal Entry,MFL98584,MFL-7902,NaN,MFL,NaN,Not Applicable,NaN,Algorithmic Offset,...,NaN,517.30,5.173000e+02,1.3782,Canada,2025,August,Missing,Not Applicable,-5.173000e+02


In [26]:
data_combined = pd.concat([add_all,offset],ignore_index=True)
data_combined

,TransactionDate,TransactionType,TransactionID_partial,DocNumber,Name,NameID,LocationRaw,FarmID,Class,Memo,...,ClassID,AmountAdj,AmountCAD,FXRate,Country,FiscalYear,Month,Location,Pillar,AmountDisplay
0,2024-11-01,Journal Entry,MFUSA12431,Not Applicable,NaN,MFUSA,-Corporate,Not Applicable,NaN,Algorithmic Reallocation,...,NaN,-20995.732764,-28936.318895,1.3782,USA,2025,November,The Pas,Not Applicable,28936.318895
1,2024-11-29,Journal Entry,MFUSA12147,Not Applicable,NaN,MFUSA,-Corporate,Not Applicable,NaN,Algorithmic Reallocation,...,NaN,20199.160993,27838.483681,1.3782,USA,2025,November,The Pas,Not Applicable,-27838.483681
2,2024-11-29,Bill,MFUSA12146,Not Applicable,Simplot Grower Solutions Billings,MFUSA216,-Corporate,Not Applicable,NaN,Algorithmic Reallocation,...,NaN,-20199.160993,-27838.483681,1.3782,USA,2025,November,The Pas,Not Applicable,27838.483681
3,2024-11-29,Vendor Credit,MFUSA12148,Not Applicable,Simplot Grower Solutions Billings,MFUSA216,-Corporate,Not Applicable,NaN,Algorithmic Reallocation,...,NaN,20995.732764,28936.318895,1.3782,USA,2025,November,The Pas,Not Applicable,-28936.318895
4,2024-12-31,Journal Entry,MFUSA12752,Not Applicable,NaN,MFUSA,-Corporate,Not Applicable,NaN,Algorithmic Reallocation,...,NaN,-796.571771,-1097.835214,1.3782,USA,2025,December,The Pas,Not Applicable,1097.835214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139898,2025-07-17,Journal Entry,MFL97048,MFL-7801,NaN,MFL,NaN,Not Applicable,NaN,Algorithmic Offset,...,NaN,5228.380000,5228.380000,1.3782,Canada,2025,July,Missing,Not Applicable,-5228.380000
139899,2025-08-05,Journal Entry,MFL98583,MFL-7901,NaN,MFL,NaN,Not Applicable,NaN,Algorithmic Offset,...,NaN,57101.360000,57101.360000,1.3782,Canada,2025,August,Missing,Not Applicable,-57101.360000
139900,2025-08-05,Journal Entry,MFL98585,MFL-7903,NaN,MFL,NaN,Not Applicable,NaN,Algorithmic Offset,...,NaN,7434.250000,7434.250000,1.3782,Canada,2025,August,Missing,Not Applicable,-7434.250000
139901,2025-08-05,Journal Entry,MFL98584,MFL-7902,NaN,MFL,NaN,Not Applicable,NaN,Algorithmic Offset,...,NaN,517.300000,517.300000,1.3782,Canada,2025,August,Missing,Not Applicable,-517.300000


In [27]:
data.AmountAdj.sum(), data.AmountCAD.sum(), data.AmountDisplay.sum()

(-72404215.07000001, -72839381.800508, 73345115.86808)

In [28]:
data_combined.AmountAdj.sum(), data_combined.AmountCAD.sum(), data_combined.AmountDisplay.sum()

(-1.862645149230957e-09, -1.862645149230957e-09, 3.725290298461914e-09)

In [29]:
regular = self.gold_pl.copy(deep=True)
regular

,TransactionDate,TransactionType,TransactionID_partial,DocNumber,Name,NameID,LocationRaw,FarmID,Class,Memo,...,ClassID,AmountAdj,AmountCAD,FXRate,Country,FiscalYear,Month,Location,Pillar,AmountDisplay
0,2024-10-01,Journal Entry,MFUSA12217,MFUSA-1405R,NaN,MFUSA,Camp 4,MFUSA6,NaN,Grain Inventory Receivable Adjustment,...,NaN,-261405.14,-3.602686e+05,1.3782,USA,2024,October,Camp 4,Grain,-3.602686e+05
1,2024-10-31,Journal Entry,MFUSA12218,MFUSA-1407,NaN,MFUSA,Camp 4,MFUSA6,NaN,Grain Inventory Receivable Adjustment,...,NaN,261405.14,3.602686e+05,1.3782,USA,2024,October,Camp 4,Grain,3.602686e+05
2,2024-11-01,Journal Entry,MFUSA12644,MFUSA-1407R,NaN,MFUSA,Camp 4,MFUSA6,NaN,Grain Inventory Receivable Adjustment,...,NaN,-261405.14,-3.602686e+05,1.3782,USA,2025,November,Camp 4,Grain,-3.602686e+05
3,2024-12-31,Journal Entry,MFUSA12645,MFUSA-1465,NaN,MFUSA,Camp 4,MFUSA6,NaN,Grain Inventory Receivable Adjustment,...,NaN,261405.14,3.602686e+05,1.3782,USA,2025,December,Camp 4,Grain,3.602686e+05
4,2024-10-01,Journal Entry,MFUSA12217,MFUSA-1405R,NaN,MFUSA,Camp 4,MFUSA6,NaN,Grain Inventory Receivable Adjustment,...,NaN,0.00,0.000000e+00,1.3782,USA,2024,October,Camp 4,Grain,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192176,2024-07-31,Journal Entry,MFL70878,MFL-5584,NaN,MFL,-Corporate,MFL4,NaN,Quarterly marketable securities adjustment,...,NaN,-16004.32,-1.600432e+04,1.3782,Canada,2024,July,Corporate,Unclassified,1.600432e+04
192177,2024-08-01,Journal Entry,MFL78125,MFL-5584R,NaN,MFL,-Corporate,MFL4,NaN,Quarterly marketable securities adjustment,...,NaN,16004.32,1.600432e+04,1.3782,Canada,2024,August,Corporate,Unclassified,-1.600432e+04
192178,2024-07-31,Journal Entry,MFL70878,MFL-5584,NaN,MFL,-Corporate,MFL4,NaN,Quarterly marketable securities adjustment,...,NaN,-1054.33,-1.054330e+03,1.3782,Canada,2024,July,Corporate,Unclassified,1.054330e+03
192179,2024-08-01,Journal Entry,MFL78125,MFL-5584R,NaN,MFL,-Corporate,MFL4,NaN,Quarterly marketable securities adjustment,...,NaN,1054.33,1.054330e+03,1.3782,Canada,2024,August,Corporate,Unclassified,-1.054330e+03


In [30]:
pl_all = pd.concat([regular,data_combined],ignore_index=True)
pl_all.groupby(["FiscalYear"]).agg({"AmountCAD":"sum"})

,AmountCAD
FiscalYear,
2018,8.115404e+06
2019,3.730036e+06
2020,1.518397e+07
2021,1.358493e+08
2022,-2.577762e+07
2023,4.926376e+07
2024,-7.046295e+07
2025,-2.030900e+08


In [31]:
regular.groupby(["FiscalYear"]).agg({"AmountCAD":"sum"})

,AmountCAD
FiscalYear,
2018,8.115404e+06
2019,3.730036e+06
2020,1.518397e+07
2021,1.358493e+08
2022,-2.577762e+07
2023,4.926376e+07
2024,-7.046295e+07
2025,-2.030900e+08


In [32]:
pl_all.to_excel(self.gold_path["finance_operational"]/"PL2.xlsx", sheet_name="Transactions", index=False)

In [153]:
self.gold_pl = pl_all

In [154]:
create_allocation_reference=False

In [155]:
def _create_additional_financial(summary:pd.DataFrame) -> pd.DataFrame:
    """ 
        this function create Gross Margin, Contribution Margin, EBITDA, Net Income financial terms in the summary table
            summary table must be broken down to fiscal year, month, location
    """
    complement_data = summary.head(0).copy(deep=True)
    for y in summary.FiscalYear.unique():
        subset_year = summary[summary["FiscalYear"] == y]
        for m in summary.Month.unique():
            subset_month = subset_year[subset_year["Month"] == m]
            for l in summary.Location.unique():
                subset_location = subset_month[subset_month["Location"] == l]
                if len(subset_location) == 0:
                    continue
                items = ["Sales Revenue", "Cost of Goods Sold", "Direct Operating Expenses", "Other Operating Revenue", "Operating Overheads", "Other Income", "Other Expense"]
                values = dict.fromkeys(items, 0)
                for i in items:
                    if i in subset_location.ProfitType.unique():
                        values[i] = subset_location.loc[subset_location["ProfitType"]==i, "AmountDisplay"].item()
                gross_margin = values["Sales Revenue"] - values["Cost of Goods Sold"]
                contribution_margin = gross_margin - values["Direct Operating Expenses"] + values["Other Operating Revenue"]
                ebitda = contribution_margin - values["Operating Overheads"]
                net_income = ebitda + values["Other Income"] - values["Other Expense"]
                row = {"FiscalYear": y, "Month": m, "Location": l}
                row_GM = row | {"ProfitType": "Gross Margin", "AmountDisplay": gross_margin}
                row_CM = row | {"ProfitType": "Contribution Margin", "AmountDisplay": contribution_margin}
                row_ebitda = row | {"ProfitType": "EBITDA", "AmountDisplay": ebitda}
                row_NI = row | {"ProfitType": "Net Income", "AmountDisplay": net_income}
                complement_data.loc[len(complement_data)] = row_GM
                complement_data.loc[len(complement_data)] = row_CM 
                complement_data.loc[len(complement_data)] = row_ebitda 
                complement_data.loc[len(complement_data)] = row_NI 
    return complement_data

In [ ]:
# prepare dfs 
data = self.gold_pl[self.gold_pl["FiscalYear"] >=  2024].copy(deep=True)
account = self.gold_acc[self.gold_acc["AccountingType"] == "Income Statement"]
# create AccID -> ProfitType mapping
id_prof_map = account.set_index("AccID")["ProfitType"]
data["ProfitType"] = data["AccID"].map(id_prof_map)
# summary by location, by pillar, by ProfitType, by Fiscal Year, by Month
summary = data.groupby(["FiscalYear", "Month", "Location", "ProfitType"]).agg({"AmountDisplay":"sum"}).reset_index(drop=False)
# prepare df for additional financial lines, e.g., Gross Margin, Net Income, ...
complement_data = _create_additional_financial(summary)
# concat two dfs
summary = pd.concat([summary, complement_data],ignore_index=True)
# assign datatype before budget summary
summary["DataType"] = "Actual"
# read processed budget transactions
budget = pd.read_csv(self.gold_path["budget"]/"OutputFile"/"all_all.csv")
budget = budget.drop(columns=["Pillar"])
budget = budget[budget["DataType"] == "Budget"]
budget = budget.loc[:,["Location", "Month", "FiscalYear", "AmountCAD", "DataType", "AccID","AccNum"]]
budget = budget.rename(columns={"AmountCAD":"AmountDisplay"})
budget = budget[~((budget["AccID"].isna())&(budget["AmountDisplay"] == 0))].reset_index(drop=True)
assert len(budget[budget["AccID"].isna()]) == 0, f"Unaccounted accounts - {budget[budget["AccID"].isna()].AccNum.unique()}"
budget = budget.drop(columns=["AccNum"])
# map ProfitType
budget["ProfitType"] = budget["AccID"].map(id_prof_map)
# budget summary
summary_budget = budget.groupby(["FiscalYear","Month","Location","ProfitType"]).agg({"AmountDisplay":"sum"}).reset_index(drop=False)
assert len(summary_budget) == len(budget.groupby(["FiscalYear","Month","Location","ProfitType"]).agg({"AmountDisplay":"sum"})), "repeat Pillar detected when summarizing budget"
# additional financial terms
complement_data_budget = _create_additional_financial(summary_budget)
# final processing
summary_budget = pd.concat([summary_budget,complement_data_budget],ignore_index=True)
summary_budget["DataType"] = "Budget"
print(f"Location missing from budget - {set(summary.Location.unique()) - set(summary_budget.Location.unique())}")
print(f"Location missing from actual - {set(summary_budget.Location.unique()) - set(summary.Location.unique())}")
# save
summary_all = pd.concat([summary, summary_budget],ignore_index=True)
summary_all.to_csv(self.gold_path["finance_operational"]/"ProfitTypeSummary2.csv", index=False) # for reclassifying accounts
summary_all.to_excel(self.gold_path["finance_operational"]/"ProfitTypeSummary2.xlsx", sheet_name="ProfitTypeSummary", index=False)

Location missing from budget - {'Billings', 'Montana (produce)', 'Colorado', 'Corporate', 'Missing', 'Cache/Fischer/Loon - DNU'}
Location missing from actual - {'Calderbank (grain)'}


In [157]:
data

,TransactionDate,TransactionType,TransactionID_partial,DocNumber,Name,NameID,LocationRaw,FarmID,Class,Memo,...,FXRate,Country,FiscalYear,Month,Location,Pillar,AmountDisplay,ProfitType,EBITDAPart,Perc
0,2024-10-01,Journal Entry,MFUSA12217,MFUSA-1405R,NaN,MFUSA,Camp 4,MFUSA6,NaN,Grain Inventory Receivable Adjustment,...,1.3844,USA,2024,October,Camp 4,Grain,-361889.275816,Sales Revenue,NaN,NaN
1,2024-10-31,Journal Entry,MFUSA12218,MFUSA-1407,NaN,MFUSA,Camp 4,MFUSA6,NaN,Grain Inventory Receivable Adjustment,...,1.3844,USA,2024,October,Camp 4,Grain,361889.275816,Sales Revenue,NaN,NaN
2,2024-11-01,Journal Entry,MFUSA12644,MFUSA-1407R,NaN,MFUSA,Camp 4,MFUSA6,NaN,Grain Inventory Receivable Adjustment,...,1.3844,USA,2025,November,Camp 4,Grain,-361889.275816,Sales Revenue,NaN,NaN
3,2024-12-31,Journal Entry,MFUSA12645,MFUSA-1465,NaN,MFUSA,Camp 4,MFUSA6,NaN,Grain Inventory Receivable Adjustment,...,1.3844,USA,2025,December,Camp 4,Grain,361889.275816,Sales Revenue,NaN,NaN
4,2024-10-01,Journal Entry,MFUSA12217,MFUSA-1405R,NaN,MFUSA,Camp 4,MFUSA6,NaN,Grain Inventory Receivable Adjustment,...,1.3844,USA,2024,October,Camp 4,Grain,0.000000,Sales Revenue,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329596,2025-07-17,Journal Entry,MFL97048,MFL-7801,NaN,MFL,NaN,Not Applicable,NaN,Algorithmic Offset,...,1.3844,Canada,2025,July,Missing,Not Applicable,-5228.380000,Direct Operating Expenses,Expense,NaN
329597,2025-08-05,Journal Entry,MFL98583,MFL-7901,NaN,MFL,NaN,Not Applicable,NaN,Algorithmic Offset,...,1.3844,Canada,2025,August,Missing,Not Applicable,-57101.360000,Direct Operating Expenses,Expense,NaN
329598,2025-08-05,Journal Entry,MFL98584,MFL-7902,NaN,MFL,NaN,Not Applicable,NaN,Algorithmic Offset,...,1.3844,Canada,2025,August,Missing,Not Applicable,-517.300000,Direct Operating Expenses,Expense,NaN
329599,2025-08-05,Journal Entry,MFL98585,MFL-7903,NaN,MFL,NaN,Not Applicable,NaN,Algorithmic Offset,...,1.3844,Canada,2025,August,Missing,Not Applicable,-7434.250000,Direct Operating Expenses,Expense,NaN


# TrailBalance - Manual Adjustments to Fert/Chem/Seed

In [32]:
self = Projects(is_dev=True)
self.gold_pl = pd.read_csv(self.gold_path["finance_operational"]/"PL.csv")
self.fx = 1.38

In [47]:
len(self.gold_pl.columns)

28

In [33]:
self.gold_pl.columns

Index(['TransactionDate', 'TransactionType', 'TransactionID_partial',
       'DocNumber', 'Name', 'NameID', 'LocationRaw', 'FarmID', 'Class', 'Memo',
       'SplitAcc', 'SplitAccID', 'Amount', 'Balance', 'AccName', 'AccID',
       'Corp', 'AccNum', 'ClassID', 'AmountAdj', 'AmountCAD', 'FXRate',
       'Country', 'FiscalYear', 'Month', 'Location', 'Pillar',
       'AmountDisplay'],
      dtype='object')

In [34]:
date = '2024-11-01'
transaction_type = 'Manual Adjustments'
memo = 'Adjustments from Trial Balance from beginning of this fiscal year'
FY = 2025
month = 'November'


In [74]:
adjustments = pd.read_csv(self.gold_path["finance_operational"]/"ManualAdjustments"/"2025.csv",dtype={"Amount":float})
adjustments.columns


Index(['AccID', 'DisplayName', 'Location', 'Amount', 'Currency', 'Pillar'], dtype='object')

In [75]:

adjustments["AmountAdj"] = -adjustments["Amount"]
adjustments["AmountCAD"] = adjustments.apply(lambda x: x["AmountAdj"] * self.fx if x["Currency"]=="USD" else x["AmountAdj"],axis=1)
adjustments["AmountDisplay"] = -adjustments["AmountCAD"]
adjustments["AccNum"] = adjustments.apply(lambda x: "".join(x["DisplayName"].split(" ")[:2]),axis=1)
adjustments

,AccID,DisplayName,Location,Amount,Currency,Pillar,AmountAdj,AmountCAD,AmountDisplay,AccNum
0,MFL270,MFL 501000 Fertilizers & Lime Expense,Hafford,1615458.00,CAD,Grain,-1615458.00,-1.615458e+06,1.615458e+06,MFL501000
1,MFL271,MFL 502000 Chemical Expense,Hafford,656218.00,CAD,Grain,-656218.00,-6.562180e+05,6.562180e+05,MFL502000
2,MFL272,MFL 503000 Seed Expense (Parent),Hafford,1662374.00,CAD,Grain,-1662374.00,-1.662374e+06,1.662374e+06,MFL503000
3,MFL270,MFL 501000 Fertilizers & Lime Expense,Kamsack,2725443.00,CAD,Grain,-2725443.00,-2.725443e+06,2.725443e+06,MFL501000
4,MFL271,MFL 502000 Chemical Expense,Kamsack,235109.00,CAD,Grain,-235109.00,-2.351090e+05,2.351090e+05,MFL502000
5,MFL272,MFL 503000 Seed Expense (Parent),Kamsack,2328018.00,CAD,Grain,-2328018.00,-2.328018e+06,2.328018e+06,MFL503000
6,MFL270,MFL 501000 Fertilizers & Lime Expense,Outlook,0.00,CAD,Produce,-0.00,-0.000000e+00,0.000000e+00,MFL501000
7,MFL271,MFL 502000 Chemical Expense,Outlook,292220.19,CAD,Produce,-292220.19,-2.922202e+05,2.922202e+05,MFL502000
8,MFL272,MFL 503000 Seed Expense (Parent),Outlook,83930.57,CAD,Produce,-83930.57,-8.393057e+04,8.393057e+04,MFL503000
9,MFL270,MFL 501000 Fertilizers & Lime Expense,Prince Albert,791728.29,CAD,Grain,-791728.29,-7.917283e+05,7.917283e+05,MFL501000


In [ ]:
row = {"TransactionDate":date, "TransactionType":transaction_type, "Memo":memo, "FiscalYear":FY, "Month":month, "FXRate": self.fx}
addition = {"Amount": 0, "AccID": "MFL2", "AmountAdj": 0, "AmountCAD": 0, "FXRate": 0, "AmountDisplay":0}

In [ ]:
addition_df = self.gold_pl.head(0).copy(deep=True)
row = {"TransactionDate":date, "TransactionType":transaction_type, "Memo":memo, "FiscalYear":FY, "Month":month, "FXRate": self.fx}
for i in range(len(adjustments)):
    entry = row | {"Amount": adjustments.loc[i,"Amount"], "AccID": adjustments.loc[i,"AccID"], "AmountAdj": adjustments.loc[i,"AmountAdj"], 
                   "AmountCAD": adjustments.loc[i,"AmountCAD"], "AmountDisplay":adjustments.loc[i,"AmountDisplay"]}
    addition_df.loc[len(addition_df)] = entry

In [46]:
addition_df

,TransactionDate,TransactionType,TransactionID_partial,DocNumber,Name,NameID,LocationRaw,FarmID,Class,Memo,...,ClassID,AmountAdj,AmountCAD,FXRate,Country,FiscalYear,Month,Location,Pillar,AmountDisplay
0,2024-11-01,Manual Adjustments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adjustments from Trial Balance from beginning ...,...,NaN,-1615458.00,-1.615458e+06,NaN,NaN,2025,November,NaN,NaN,1.615458e+06
1,2024-11-01,Manual Adjustments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adjustments from Trial Balance from beginning ...,...,NaN,-656218.00,-6.562180e+05,NaN,NaN,2025,November,NaN,NaN,6.562180e+05
2,2024-11-01,Manual Adjustments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adjustments from Trial Balance from beginning ...,...,NaN,-1662374.00,-1.662374e+06,NaN,NaN,2025,November,NaN,NaN,1.662374e+06
3,2024-11-01,Manual Adjustments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adjustments from Trial Balance from beginning ...,...,NaN,-2725443.00,-2.725443e+06,NaN,NaN,2025,November,NaN,NaN,2.725443e+06
4,2024-11-01,Manual Adjustments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adjustments from Trial Balance from beginning ...,...,NaN,-235109.00,-2.351090e+05,NaN,NaN,2025,November,NaN,NaN,2.351090e+05
5,2024-11-01,Manual Adjustments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adjustments from Trial Balance from beginning ...,...,NaN,-2328018.00,-2.328018e+06,NaN,NaN,2025,November,NaN,NaN,2.328018e+06
6,2024-11-01,Manual Adjustments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adjustments from Trial Balance from beginning ...,...,NaN,-0.00,-0.000000e+00,NaN,NaN,2025,November,NaN,NaN,0.000000e+00
7,2024-11-01,Manual Adjustments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adjustments from Trial Balance from beginning ...,...,NaN,-292220.19,-2.922202e+05,NaN,NaN,2025,November,NaN,NaN,2.922202e+05
8,2024-11-01,Manual Adjustments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adjustments from Trial Balance from beginning ...,...,NaN,-83930.57,-8.393057e+04,NaN,NaN,2025,November,NaN,NaN,8.393057e+04
9,2024-11-01,Manual Adjustments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adjustments from Trial Balance from beginning ...,...,NaN,-791728.29,-7.917283e+05,NaN,NaN,2025,November,NaN,NaN,7.917283e+05


# budget seasonality

## isolate last year's transactions

In [75]:
transactions = self._budget_get_transactions()
transactions["TransactionDate"] = pd.to_datetime(transactions["TransactionDate"])
transactions.FiscalYear.value_counts()

FiscalYear
2024    52248
2025    44523
Name: count, dtype: int64

In [76]:
self.currentFY

2025

In [77]:
transactions1 = transactions[transactions["FiscalYear"]==self.currentFY-1]
transactions1.FiscalYear.value_counts()

FiscalYear
2024    52248
Name: count, dtype: int64

## summarize for each account and amount of transactions per month

In [78]:
transactions1.columns

Index(['TransactionDate', 'TransactionType', 'TransactionID_partial',
       'DocNumber', 'Name', 'NameID', 'LocationRaw', 'FarmID', 'Class', 'Memo',
       'SplitAcc', 'SplitAccID', 'Amount', 'Balance', 'AccName', 'AccID',
       'Corp', 'AccNum', 'ClassID', 'AmountAdj', 'AmountCAD', 'FXRate',
       'Country', 'FiscalYear', 'Month', 'Location', 'Pillar',
       'AmountDisplay'],
      dtype='object')

In [79]:
trend = transactions1.groupby(["AccID","Month", "Location"]).agg({"AmountCAD":"sum"}).reset_index(drop=False)
total = transactions1.groupby(["AccID", "Location"]).agg({"AmountCAD": "sum"}).reset_index(drop=False)

In [80]:
trend2 = pd.merge(trend.rename(columns={"AmountCAD":"MonthlyCAD"}), total.rename(columns={"AmountCAD": "YearlyCAD"}),
                  on = ["AccID", "Location"], how="left")
trend2["Monthly%"] = np.round(trend2["MonthlyCAD"] / trend2["YearlyCAD"] * 100, 2 )
trend2

,AccID,Month,Location,MonthlyCAD,YearlyCAD,Monthly%
0,MFAZ101,April,Arizona (corporate),-4604.507720,-67231.468484,6.85
1,MFAZ101,April,Arizona (grain),-95.984665,-5581.026021,1.72
2,MFAZ101,April,Arizona (produce),-2384.209467,-31108.453237,7.66
3,MFAZ101,August,Arizona (corporate),-2405.308178,-67231.468484,3.58
4,MFAZ101,August,Arizona (grain),-2824.429731,-5581.026021,50.61
...,...,...,...,...,...,...
10466,NexGen94,March,NexGen,-6206.400000,-48158.310000,12.89
10467,NexGen94,May,NexGen,-2316.470000,-48158.310000,4.81
10468,NexGen94,November,NexGen,-5131.370000,-48158.310000,10.66
10469,NexGen94,October,NexGen,-4304.760000,-48158.310000,8.94


In [81]:
budget = pd.read_csv(self.gold_path["budget"]/"OutputFile"/"budget_all.csv")
budget
# budget = budget.loc[:,["Location","AccNum","TotalAmountCAD"]]

,Location,Category,AccFull,SheetRef,Month,Formula,TotalAmountCAD,AmountCAD,AccRef,ReferenceYear,FiscalYear,AccNum,AccName,DataType
0,Regina,Fertilizer,MFL 501000 Fertilizers & Lime Expense,Input Budget,November,/4,4.154073e+06,1.038518e+06,NaN,NaN,2025,MFL501000,Fertilizers & Lime Expense,Budget
1,Raymore,Fertilizer,MFL 501000 Fertilizers & Lime Expense,Input Budget,November,/4,1.460318e+06,3.650795e+05,NaN,NaN,2025,MFL501000,Fertilizers & Lime Expense,Budget
2,Prince Albert,Fertilizer,MFL 501000 Fertilizers & Lime Expense,Input Budget,November,/2,2.496507e+06,1.248254e+06,NaN,NaN,2025,MFL501000,Fertilizers & Lime Expense,Budget
3,The Pas,Fertilizer,MFL 501000 Fertilizers & Lime Expense,Input Budget,November,*0,3.008469e+06,0.000000e+00,NaN,NaN,2025,MFL501000,Fertilizers & Lime Expense,Budget
4,Hafford,Fertilizer,MFL 501000 Fertilizers & Lime Expense,Input Budget,November,*0.47,4.213666e+06,1.980423e+06,NaN,NaN,2025,MFL501000,Fertilizers & Lime Expense,Budget
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51271,Seeds USA,Wages and benefits - overhead,MSUSA 707470 AZ State Unemployment Tax,NaN,October,NaN,1.523671e+02,2.103733e+02,Copy,NaN,2025,MSUSA707470,AZ State Unemployment Tax,Budget
51272,Seeds USA,Wages and benefits - overhead,MSUSA 707500 Wages - Other Insurance,NaN,October,NaN,1.522075e+02,2.101529e+02,Copy,NaN,2025,MSUSA707500,Wages - Other Insurance,Budget
51273,Seeds USA,Wages and benefits - overhead,MSUSA 707550 Payroll Service Fees,NaN,October,NaN,2.700252e+02,3.728238e+02,Copy,NaN,2025,MSUSA707550,Payroll Service Fees,Budget
51274,Seeds USA,Amortization expense,MSUSA 701200 Depreciation Expense Building,NaN,October,NaN,1.291791e+04,1.783576e+04,Copy,NaN,2025,MSUSA701200,Depreciation Expense Building,Budget


In [82]:
budget[budget["Category"].str.contains("amortization",case=False)]

,Location,Category,AccFull,SheetRef,Month,Formula,TotalAmountCAD,AmountCAD,AccRef,ReferenceYear,FiscalYear,AccNum,AccName,DataType
5451,Regina,Amortization expense,MFL 581000 Depreciation Expense - Production,NaN,November,/12*1.05,2.385314e+06,208714.961875,MFL 581000 Depreciation Expense - Production,FY-1,2025,MFL581000,Depreciation Expense - Production,Budget
5452,Raymore,Amortization expense,MFL 581000 Depreciation Expense - Production,NaN,November,/12*1.05,1.519977e+06,132997.969125,MFL 581000 Depreciation Expense - Production,FY-1,2025,MFL581000,Depreciation Expense - Production,Budget
5453,Prince Albert,Amortization expense,MFL 581000 Depreciation Expense - Production,NaN,November,/12*1.05,1.596275e+06,139674.055500,MFL 581000 Depreciation Expense - Production,FY-1,2025,MFL581000,Depreciation Expense - Production,Budget
5454,The Pas,Amortization expense,MFL 581000 Depreciation Expense - Production,NaN,November,/12*1.05,4.086405e+05,35756.045500,MFL 581000 Depreciation Expense - Production,FY-1,2025,MFL581000,Depreciation Expense - Production,Budget
5455,Hafford,Amortization expense,MFL 581000 Depreciation Expense - Production,NaN,November,/12*1.05,2.388373e+06,208982.663750,MFL 581000 Depreciation Expense - Production,FY-1,2025,MFL581000,Depreciation Expense - Production,Budget
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51195,Seeds,Amortization expense,MSL 569500 Depreciation Expense - Machinery,NaN,October,NaN,2.406371e+04,24063.705000,Copy,NaN,2025,MSL569500,Depreciation Expense - Machinery,Budget
51196,Seeds,Amortization expense,MSL 590000 Depreciation Expense - Buildings,NaN,October,NaN,5.302937e+03,5302.936667,Copy,NaN,2025,MSL590000,Depreciation Expense - Buildings,Budget
51234,NexGen,Amortization expense,NexGen 569500 Depreciation Expense - Machinery,NaN,October,NaN,3.232178e+03,3232.178425,Copy,NaN,2025,NexGen569500,Depreciation Expense - Machinery,Budget
51235,NexGen,Amortization expense,NexGen 590000 Depreciation Expense - Buildings,NaN,October,NaN,3.184971e+03,3184.970917,Copy,NaN,2025,NexGen590000,Depreciation Expense - Buildings,Budget


In [83]:
budget2 = budget[budget["Formula"] == "/12"].copy(deep=True)
budget2

,Location,Category,AccFull,SheetRef,Month,Formula,TotalAmountCAD,AmountCAD,AccRef,ReferenceYear,FiscalYear,AccNum,AccName,DataType
3770,Regina,Grain - cash settlements,MFL 505000 Drying/Elevator Adjustments,NaN,November,/12,2.517542e+05,20979.514167,MFL 505000 Drying/Elevator Adjustments,FY-1,2025,MFL505000,Drying/Elevator Adjustments,Budget
3771,Raymore,Grain - cash settlements,MFL 505000 Drying/Elevator Adjustments,NaN,November,/12,2.635137e+04,2195.947500,MFL 505000 Drying/Elevator Adjustments,FY-1,2025,MFL505000,Drying/Elevator Adjustments,Budget
3772,Prince Albert,Grain - cash settlements,MFL 505000 Drying/Elevator Adjustments,NaN,November,/12,3.777503e+04,3147.919167,MFL 505000 Drying/Elevator Adjustments,FY-1,2025,MFL505000,Drying/Elevator Adjustments,Budget
3773,The Pas,Grain - cash settlements,MFL 505000 Drying/Elevator Adjustments,NaN,November,/12,4.029989e+04,3358.324167,MFL 505000 Drying/Elevator Adjustments,FY-1,2025,MFL505000,Drying/Elevator Adjustments,Budget
3774,Hafford,Grain - cash settlements,MFL 505000 Drying/Elevator Adjustments,NaN,November,/12,4.833570e+03,402.797500,MFL 505000 Drying/Elevator Adjustments,FY-1,2025,MFL505000,Drying/Elevator Adjustments,Budget
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48453,Outlook,Crop insurance,MFL 581503 Crop & Hail Insurance,NaN,October,/12,5.801500e+05,48345.833333,Fixed,NaN,2025,MFL581503,Crop & Hail Insurance,Budget
48456,Arizona (produce),Crop insurance,MFAZ 506305 Crop Insurance Expense,NaN,October,/12,2.360093e+05,19667.438000,Fixed,NaN,2025,MFAZ506305,Crop Insurance Expense,Budget
48462,Arizona (produce),Freight and trucking,MFAZ 592001 Freight & Trucking Expense,NaN,October,/12,2.455298e+06,204608.186333,Fixed,NaN,2025,MFAZ592001,Freight & Trucking Expense,Budget
48465,Arizona (produce),Supplies,MFAZ 593000 Job Supplies,NaN,October,/12,9.533470e+05,79445.585858,Fixed,NaN,2025,MFAZ593000,Job Supplies,Budget


In [84]:
budget2.AmountCAD.sum()

55534039.80643467

In [85]:
self._extract_accnum_accid()
self.acc_map

AccNum
MFUSA210010     MFUSA65
MFUSA120100    MFUSA203
MFUSA120000     MFUSA33
MFUSA120040     MFUSA35
MFUSA215500     MFUSA69
                 ...   
MFL571012        MFL431
MFL586006        MFL351
MFL586012        MFL355
MFL574200        MFL312
MFL571005        MFL297
Name: AccID, Length: 3094, dtype: object

In [86]:
budget2["AccID"] = budget2["AccNum"].map(self.acc_map)
budget2

,Location,Category,AccFull,SheetRef,Month,Formula,TotalAmountCAD,AmountCAD,AccRef,ReferenceYear,FiscalYear,AccNum,AccName,DataType,AccID
3770,Regina,Grain - cash settlements,MFL 505000 Drying/Elevator Adjustments,NaN,November,/12,2.517542e+05,20979.514167,MFL 505000 Drying/Elevator Adjustments,FY-1,2025,MFL505000,Drying/Elevator Adjustments,Budget,MFL275
3771,Raymore,Grain - cash settlements,MFL 505000 Drying/Elevator Adjustments,NaN,November,/12,2.635137e+04,2195.947500,MFL 505000 Drying/Elevator Adjustments,FY-1,2025,MFL505000,Drying/Elevator Adjustments,Budget,MFL275
3772,Prince Albert,Grain - cash settlements,MFL 505000 Drying/Elevator Adjustments,NaN,November,/12,3.777503e+04,3147.919167,MFL 505000 Drying/Elevator Adjustments,FY-1,2025,MFL505000,Drying/Elevator Adjustments,Budget,MFL275
3773,The Pas,Grain - cash settlements,MFL 505000 Drying/Elevator Adjustments,NaN,November,/12,4.029989e+04,3358.324167,MFL 505000 Drying/Elevator Adjustments,FY-1,2025,MFL505000,Drying/Elevator Adjustments,Budget,MFL275
3774,Hafford,Grain - cash settlements,MFL 505000 Drying/Elevator Adjustments,NaN,November,/12,4.833570e+03,402.797500,MFL 505000 Drying/Elevator Adjustments,FY-1,2025,MFL505000,Drying/Elevator Adjustments,Budget,MFL275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48453,Outlook,Crop insurance,MFL 581503 Crop & Hail Insurance,NaN,October,/12,5.801500e+05,48345.833333,Fixed,NaN,2025,MFL581503,Crop & Hail Insurance,Budget,MFL324
48456,Arizona (produce),Crop insurance,MFAZ 506305 Crop Insurance Expense,NaN,October,/12,2.360093e+05,19667.438000,Fixed,NaN,2025,MFAZ506305,Crop Insurance Expense,Budget,MFAZ1150000002
48462,Arizona (produce),Freight and trucking,MFAZ 592001 Freight & Trucking Expense,NaN,October,/12,2.455298e+06,204608.186333,Fixed,NaN,2025,MFAZ592001,Freight & Trucking Expense,Budget,MFAZ64
48465,Arizona (produce),Supplies,MFAZ 593000 Job Supplies,NaN,October,/12,9.533470e+05,79445.585858,Fixed,NaN,2025,MFAZ593000,Job Supplies,Budget,MFAZ15


In [87]:
budget2[budget2["AccID"].isna()].AmountCAD.sum()

0.0

In [95]:
unique_budget_acc = list(set(budget2.AccID.unique()) - set(trend2.AccID.unique()))
len(unique_budget_acc), len(budget2.AccID.unique())

(126, 420)

In [92]:
budget2[budget2["AccID"].isin(unique_budget_acc)].AmountCAD.sum()

236009.25599999996

In [97]:
self.operation_acc[self.operation_acc["AccID"].isin(unique_budget_acc)].OperationProfType.isna().sum()

63

# HP class

In [72]:
self = Job()

In [73]:
self.raw_path["Auth"]["Harvest Profit"]

WindowsPath('c:/Users/ZheRao/OneDrive - Monette Farms/Monette Farms Team Site - Innovation Projects/Production/Database/Bronze/Authentication/Harvest Profit')

In [74]:
self.json_download_path

WindowsPath('c:/Users/ZheRao/OneDrive - Monette Farms/Desktop/Work Files/Projects/5 - HP Data')

In [75]:
def purge_folder():
    """ 
        This function will clear all files in the folder that is used to store emailed links
            called before pipeline and after pipeline to ensure no duplicated download or errors in sending download requests
    """
    for json_path in os.listdir(self.json_download_path):
        Path.unlink(self.json_download_path/json_path, missing_ok=True)

In [76]:
purge_folder()

## Request link - trigger emails

In [77]:
s = requests.Session()

### login

In [78]:
BASE = "https://www.harvestprofit.com"
LOGIN_GET = f"{BASE}/users/sign_in"

In [79]:
def _extract_csrf_and_action(html: str):
    soup = BeautifulSoup(html, "html.parser")
    # auth token can appear as a hidden input or in a meta tag
    token = None 
    # find correct form in the login page to extract tokens and action for sending the login request
    for form in soup.find_all("form"):
        action = (form.get("action") or "")
        if form.find("input", {"type": "password"}) or "/users/sign_in" in action:
            login_form = form
            break
    if not login_form:
        raise RuntimeError("Couldn't find the login form.")
    token = (login_form.find("input", {"name": "authenticity_token"}) or {}).get("value")
    if not token:
        meta = soup.find("meta", {"name": "csrf-token"})
        token = meta.get("content") if meta else None
    if not token:
        raise RuntimeError("Missing CSRF token.")
    return token, action

In [80]:
# extract info from login page
login = s.get(LOGIN_GET)
login.raise_for_status()
# retrieve token and action for login request
token, action = _extract_csrf_and_action(login.text)
# conpose precise endpoint for login request
post_url = urljoin(BASE, action)

In [81]:
# extract credentials locally 
with open(self.raw_path["Auth"]["Harvest Profit"]/"info.json", "r") as f:
    credentials = json.load(f)
email = credentials["Credentials"]["email"]
passwrd = credentials["Credentials"]["password"]

In [82]:
# send login request
payload = {
    "authenticity_token": token,
    "user[email]": email,
    "user[password]": passwrd
}
headers = {
    "Origin": BASE,
    "Referer": LOGIN_GET
}
login = s.post(post_url, data=payload, headers=headers, allow_redirects=True)
login.raise_for_status()
if login.status_code in ["200", 200]:
    print("Login Successful")

Login Successful


### switch accounts and extrieve request-email-download link

In [83]:
BASE = "https://www.harvestprofit.com"
hp_locations = list(credentials["SwitchAcc"].keys())
hp_locations

['Billings',
 'BoxElder',
 'SwiftCurrent',
 'Eddystone',
 'ThePas',
 'Hafford',
 'Kamsack',
 'PA',
 'Raymore',
 'Regina',
 'Airdrie',
 'Outlook']

In [84]:
for l in hp_locations:
    # switch account 
    print(f"activating {l}")
    ACTIVATE = credentials["SwitchAcc"][l]
    r = s.get(ACTIVATE, headers={"Referer": f"{BASE}/accounts"}, allow_redirects=True)
    r.raise_for_status()
    # get relevant information from updated load page for sending the request for email
    soup = BeautifulSoup(r.text)
    script = soup.find("script", {"class": "js-react-on-rails-component", "data-component-name": "Application", "type": "application/json"})
    data = json.loads(script.text)
    access_jwt, refresh_jwt = data.get("token"), data.get("refresh_token")
    print(f"Page loaded for {data["entity"]["name"]}")
    # sending the request for email containing data extraction link
    url = f"{BASE}/api/v3/grain_inventory/loads/export_all"
    year_ids = [y['id'] for y in data["years"]]
    params = [("year_ids[]", str(y)) for y in year_ids]
    LOADS_URL = "https://www.harvestprofit.com/78691/grain_inventory/loads"
    headers = {
        "Accept": "text/csv, application/json, text/plain, */*",
        "Referer": LOADS_URL,
        "authorization": access_jwt
    }
    resp = s.get(url,params = params, headers=headers)
    resp.raise_for_status()

activating Billings
Page loaded for MFB Billings - US
activating BoxElder
Page loaded for MFBE Box Elder - US
activating SwiftCurrent
Page loaded for MFS Swift Current - CA
activating Eddystone
Page loaded for MFE Eddystone - CA
activating ThePas
Page loaded for MFPAS The Pas - CA
activating Hafford
Page loaded for MFH Hafford - CA
activating Kamsack
Page loaded for MFK Kamsack - CA
activating PA
Page loaded for MFPA  Prince Albert - CA
activating Raymore
Page loaded for MFRAY Raymore - CA
activating Regina
Page loaded for MFR Regina - CA
activating Airdrie
Page loaded for MFAIR Airdrie - CA
activating Outlook
Page loaded for MFO Outlook - CA


## HP Extract from Links - wait 180 seconds

In [87]:

os.listdir(self.json_download_path)

['20250926_AAMkAGQ0M2FmN2ZhLTU3ZDQtNGYyOS1hNDIwLTk0ZTJiMTQ4YmJkOQBGAAAAAABcpKS7KOmPR5tmeIn3LzxlBwBCUPpxjwcWSZPqpurZJFX4AAB1NsaaAABCUPpxjwcWSZPqpurZJFX4AACLVR2HAAA=.json',
 '20250926_AAMkAGQ0M2FmN2ZhLTU3ZDQtNGYyOS1hNDIwLTk0ZTJiMTQ4YmJkOQBGAAAAAABcpKS7KOmPR5tmeIn3LzxlBwBCUPpxjwcWSZPqpurZJFX4AAB1NsaaAABCUPpxjwcWSZPqpurZJFX4AACLVR2IAAA=.json',
 '20250926_AAMkAGQ0M2FmN2ZhLTU3ZDQtNGYyOS1hNDIwLTk0ZTJiMTQ4YmJkOQBGAAAAAABcpKS7KOmPR5tmeIn3LzxlBwBCUPpxjwcWSZPqpurZJFX4AAB1NsaaAABCUPpxjwcWSZPqpurZJFX4AACLVR2JAAA=.json',
 '20250926_AAMkAGQ0M2FmN2ZhLTU3ZDQtNGYyOS1hNDIwLTk0ZTJiMTQ4YmJkOQBGAAAAAABcpKS7KOmPR5tmeIn3LzxlBwBCUPpxjwcWSZPqpurZJFX4AAB1NsaaAABCUPpxjwcWSZPqpurZJFX4AACLVR2KAAA=.json',
 '20250926_AAMkAGQ0M2FmN2ZhLTU3ZDQtNGYyOS1hNDIwLTk0ZTJiMTQ4YmJkOQBGAAAAAABcpKS7KOmPR5tmeIn3LzxlBwBCUPpxjwcWSZPqpurZJFX4AAB1NsaaAABCUPpxjwcWSZPqpurZJFX4AACLVR2LAAA=.json',
 '20250926_AAMkAGQ0M2FmN2ZhLTU3ZDQtNGYyOS1hNDIwLTk0ZTJiMTQ4YmJkOQBGAAAAAABcpKS7KOmPR5tmeIn3LzxlBwBCUPpxjwcWSZPqpurZJFX4AAB1NsaaAABCUPpxjwcWSZPqpu

In [88]:
assert len(os.listdir(self.json_download_path)) == 11, f"number of files not matching - should be 11, got {len(os.listdir(self.json_download_path))}"

In [89]:
df_all = pd.DataFrame()
for json_path in os.listdir(self.json_download_path):
    # open link file
    with open(self.json_download_path/json_path, "r") as f:
        link = json.load(f)
    # send download request
    r = requests.get(link['url'])
    # extract data from request results
    df = pd.read_csv(
        StringIO(r.text),       # treat the string like a file
        skipinitialspace=True,  # trims the leading space in " Lbs"
        parse_dates=['date'],   # pase the datetime column
        date_format="%m/%d/%Y %I:%M %P",    # speeds up parsing for this format
        dtype={'harvest_profit_id': 'Int64', 'crop_year': 'Int64'},
        na_values=['']          # turn empty quotes into NaN
    )
    print(f"Processed - {df.loc[0,"entity_share"]}")
    # append results 
    df_all = pd.concat([df_all,df], ignore_index=True)


Processed - MFE Eddystone - CA
Processed - MFPAS The Pas - CA
Processed - MFRAY Raymore - CA
Processed - MFAIR Airdrie - CA
Processed - MFPA  Prince Albert - CA
Processed - MFK Kamsack - CA
Processed - MFO Outlook - CA
Processed - MFR Regina - CA
Processed - MFH Hafford - CA
Processed - MFS Swift Current - CA
Processed - MFB Billings - US


In [91]:
s.close()

In [92]:
purge_folder()

In [20]:
df_all.columns

Index(['crop', 'from', 'to', 'amount', 'accepted_amount', 'amount_unit',
       'details', 'harvest_profit_id', 'date', 'truck', 'entity_share',
       'crop_year', 'moisture', 'test_weight', 'crop_moisture', 'dry_amount',
       'weight', 'bushels', 'shrink', 'dockage_percent', 'protein_content',
       'foreign_matter', 'damage', 'HVK', 'grade', 'created_by', 'contract'],
      dtype='object')

In [21]:
df_all.entity_share.value_counts()

entity_share
MFB Billings - US           3738
MFS Swift Current - CA      1769
MFH Hafford - CA            1430
MFK Kamsack - CA            1369
MFR Regina - CA             1259
MFPAS The Pas - CA          1068
MFPA  Prince Albert - CA     765
MFRAY Raymore - CA           537
MFE Eddystone - CA           267
MFO Outlook - CA             252
MFAIR Airdrie - CA           212
Name: count, dtype: int64

In [93]:
df_all.to_csv(self.silver_path["Delivery"]["HP"]/f"Loads_{self.today.year}_{self.today.month}.csv", index=False)

In [72]:
df_all.isna().sum()

crop                   157
from                  3613
to                    1475
amount                   0
accepted_amount          0
amount_unit              0
details               9865
harvest_profit_id        0
date                     0
truck                 6221
entity_share             0
crop_year                0
moisture                 0
test_weight              0
crop_moisture          157
dry_amount             157
weight                 157
bushels                157
shrink                   0
dockage_percent          0
protein_content       9805
foreign_matter        9805
damage                9805
HVK                  11840
grade                11840
created_by            9860
contract              9911
dtype: int64

In [90]:
df_all

,crop,from,to,amount,accepted_amount,amount_unit,details,harvest_profit_id,date,truck,...,bushels,shrink,dockage_percent,protein_content,foreign_matter,damage,HVK,grade,created_by,contract
0,Barley,Tamrack,NaN,0.0,0.0,Lbs,NaN,982844,08/05/2025 06:50 PM,NaN,...,0.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Barley,Tamrack,NaN,0.0,0.0,Lbs,NaN,982957,08/05/2025 08:45 PM,NaN,...,0.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Barley,Tamrack,NaN,0.0,0.0,Lbs,NaN,982967,08/05/2025 08:57 PM,NaN,...,0.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Barley,Tamrack,NaN,0.0,0.0,Lbs,NaN,982988,08/05/2025 09:40 PM,NaN,...,0.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Barley,Tamrack,NaN,59208.9,59208.9,Lbs,NaN,983835,08/07/2025 03:06 PM,NaN,...,1233.52,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14461,Wheat (Hard Red Winter),Bag 27,Viterra Huntley,59460.0,59460.0,Lbs,Ticket 0016810,1069555,09/10/2025 04:40 PM,03 KW T880,...,991.00,0.0,0.0,0.0,0.0,0.0,NaN,NaN,kobusaustin@hotmail.co.za,B001150 - Camp 4 • December Basis
14462,Wheat (Hard Red Winter),Bag 27,Viterra Huntley,51360.0,51360.0,Lbs,Ticket 0016794,1069559,09/08/2025 04:41 PM,03 KW T880,...,856.00,0.0,0.0,0.0,0.0,0.0,NaN,NaN,kobusaustin@hotmail.co.za,B001150 - Camp 4 • December Basis
14463,Wheat (Hard Red Winter),Bag 27,Viterra Huntley,66740.0,66740.0,Lbs,Ticket 0016786,1069566,09/08/2025 04:42 PM,03 KW T880,...,1112.33,0.0,0.0,0.0,0.0,0.0,NaN,NaN,kobusaustin@hotmail.co.za,B001150 - Camp 4 • December Basis
14464,Wheat (Hard Red Winter),Bag 27,Viterra Huntley,57680.0,57680.0,Lbs,Ticket 0016780,1069568,09/08/2025 04:43 PM,03 KW T880,...,961.33,0.0,0.0,0.0,0.0,0.0,NaN,NaN,kobusaustin@hotmail.co.za,B001150 - Camp 4 • December Basis


In [24]:
df_all.amount_unit.value_counts()

amount_unit
Lbs    12627
Bu        39
Name: count, dtype: int64

In [39]:
df_all.contract.value_counts()

contract
B025031 - Fly Creek  • September Basis    465
B025030 - Camp 4 • September Basis        403
5179278 • December Basis                  190
8003535 • September Sale                  146
5180529 • September Sale                  143
5179267 • December Basis                  127
5180530 • September Sale                  119
5208132 • September Basis                  85
5179270 • December Basis                   57
59720 • December Basis                     54
504482 • August Sale                       46
753138 • November Basis                    37
5179272 • December Basis                   36
2100430543-000010 • November Basis         29
5200624 • November Basis                   26
5191316 • September Basis                  18
NCC - 8003814 • November Basis             15
B001151 - Fly Creek  • December Basis       5
5209439 • November Basis                    1
5208122 • November Basis                    1
Name: count, dtype: int64